In [ ]:
Данный ноутбук решает задачу детектирования астрономических объектов:
0.На данный момент точно работает только на srg из пользователя kolosovi
1.Задать параметры (или использовать мои) в configs. 
2.Запустить ячейки в секция 3 и 4. Там код модели и обработки данных
4.Запустить код в секции 5: настроить размер датасета, пути ко входным данным.

# Import all libraries
Most are in libs file

In [1]:
import sys
sys.path.insert(0, '/home/kolosovi/work/all_notebooks/')
sys.path.insert(0, '/home/kolosovi')

import erospy
from libs import *


importlib.reload(erospy)
from notebook.services.config import ConfigManager
cm = ConfigManager().update('notebook', {'limit_output': 100000000000000000})
%matplotlib inline
sns.set(color_codes=True)
warnings.simplefilter('ignore')

Using TensorFlow backend.


# Create config class
All file path work, preprosessing and detection settings are here

In [67]:
class Config():
    def __init__(self, data_type, dataset_type, remove_old = False):
        #data
        self.data_type = data_type
        
        #Depending on the type of dataset - select a path for data. Assume that all data is there:
        # -photon files
        # -simput files
        
        #train - train dataset
        #test - test dataset
        #cluster - cluster dataset
        if data_type == 'train':
            self.data_path = '/srg/a1/work/kate/erosita/agn/2.4.2/1e17_for_mesh/'
            self.noize_path = '/srg/a1/work/kate/erosita/bkg/for_mesh'
            self.simput_file = os.path.join(self.data_path , '%i_agncat.simput')
        elif data_type == 'test':
            self.data_path = '/srg/a1/work/kate/erosita/agn/2.4.2/1e17/' #0_agncat.simput'
            self.noize_path = '/srg/a1/work/kate/erosita/bkg/for_sim/' #bkg_?.fits'
            self.simput_file = os.path.join(self.data_path , '%i_agncat.simput')
        elif data_type == 'cluster':
            self.data_path = '/srg/a1/work/kate/erosita/clusters/for_rodion' 
            self.noize_path = '/srg/a1/work/kate/erosita/bkg/for_sim/' #bkg_?.fits'
            self.simput_file = os.path.join(self.data_path , '%i_clusters.simput') 

        #Now when specified directories, it's time to define file name patterns in this directories
        self.ph_ero_file = os.path.join(self.data_path , '%i_4yr_ccdall_eroevt.fits')
        self.ph_evt_file = os.path.join(self.data_path , '%i_4yr_ccdall_evt.fits')
        self.noize_file = os.path.join(self.noize_path , 'bkg_%i.fits')
        
        #Just for clusters
        self.cl_data_path = '/srg/a1/work/kate/erosita/clusters/for_rodion'
        self.cl_simput_file = os.path.join(self.cl_data_path , '%i_clusters.simput') 
        self.cl_ph_ero_file = os.path.join(self.cl_data_path , '%i_4yr_ccdall_eroevt.fits')
        self.cl_ph_evt_file = os.path.join(self.cl_data_path , '%i_4yr_ccdall_evt.fits')
        
        
        self.work_path = '/home/kolosovi/work'
        
        
        #path for esass results
        self.esass_res_path = '/srg/a1/work/medvedev/agn_eSASS_190220_noEXT'
        self.nn_res_path = os.path.join(self.work_path, 'data/my_output/final')
        self.nn_res_path_noize = os.path.join(self.work_path, 'data/my_output/final_noize')
        self.nn_res_path_nonoize = os.path.join(self.work_path, 'data/my_output/final_nonoize')
        
        #data input for nn paths
        self.model_path = os.path.join(self.work_path, 'data/models/') 
        self.exposure_file = os.path.join(self.work_path, 'data/exposure/exp.fits')
        self.telescope_path = os.path.join(self.work_path, 'data/telescope/')
        self.wcs_file = os.path.join(self.work_path, 'data/projection_image.fits')
        self.valid_frames_file = os.path.join(self.work_path, 'data/valid_frames.npy')
        self.all_frames_file = os.path.join(self.work_path, 'data/all_frames.npy')
        self.boxlist_file = os.path.join(self.esass_res_path, 'sim_0/BoxDetSourceListL.fits')
        
        #Dataset
        self.save_train_data = False
        self.dataset_type = dataset_type
        self.dataset_path = os.path.join(f'/data/kolosovi/dataset_{self.dataset_type}/')
        
        if dataset_type == 'tmp' or remove_old:
            if os.path.exists(self.dataset_path):
                print(f'Erasing {self.dataset_path}')
                shutil.rmtree(self.dataset_path)
            os.makedirs(self.dataset_path)
        
        #create dir for dataset if not exists 
        d = os.path.dirname(self.dataset_path)
        if not os.path.exists(d):
                os.makedirs(d)
        
        self.dataset_file = os.path.join(self.dataset_path , 'df_%d_frame_%d.npy')
        self.save_ims = False,

        #--------------Preprosessing params-------------------------------------------
        #KDE
        self.border_KDE = False          
        self.high_border_weight = False 
        self.KDE_bandwidth = 7
        self.p_mask = 0.0016
            #0.0016, may be i changed something. but it's too big now.
            #0.0007 also may be to big
            #old 0.0003 value leads to too big blobs in prediction 

        #PHOTONS
        self.remove_null_pha = False
        self.photon_limit_cut = False
        self.photon_limit = 15 #Delete object if has < photon_limit
        self.conf_limit = 2.3 * (10**-15)
        self.min_flux = 2.3 * (10**-15) #confusion limit, objects with flux<min_flux - noize
        #flux_limit = [2.3 * 10**-15, 1.1 * 10**-14], #, 4.4 * 10**-14],
        self.flux_limit = [2.3 * (10**-15)]

        #INPUT IMAGE
        self.channels_N = 32 #always pow of 2
        self.full_energy_channel = False
        self.pha_channels_N = 8
        self.offset_channel = True            
        self.offset_channels_N = 4
        #max_PHA = 960, #for channel size
        self.sky_size = 2680 #cut df to this size by X,Y
        self.P0 = 0 #starting from P0
        self.object_weight = 1
        self.add_noize = True
        self.n_src = 100 #for colors

        #SOURCES
        self.correct_centers = False
        self.esaas_thres_list = [1,2,3,8,10,13,16,20,30]
        self.unet_thres_list_big = [0.07179278, 0.22292863, 0.40038235]
        self.unet_thres_list = [0.5,0.55,0.67,0.76,0.82, 0.89, 0.94]

        #--------------Postprosessing params-------------------------------------------
        self.verbose = False
        self.blob_thres = 8
        self.gauss_output = False
        self.gauss_size = 3
        
        #BOXLIST PARAMS
        #for blobs and coordinates
        self.save_boxlist_k = 3 #3x row duplicating
        self.min_object_p = 0.2 #n't take to blob if lower
        self.min_center_p = 0.8
        self.min_blob_pixels = 35 #remove blob if it's less then 10 pixels
        self.min_photons_in_blob = 14
        #for noize
        self.mean_ECF = 1.87 * 1e-12 #for flux conversion
        self.exp_k = 1.5 * 2.561900e-10 / 3.228494e-13  #just divided original flux and experimental flux
        self.noize_bound = 0.2 # <0.2 this is fucking noize - need it to count noize mean across image
        

        #args['KDE_bandwidth'] = 3
        #args['p_mask'] = 0.0016 

        
    def __getitem__(self,key):
        return getattr(self,key)
    def __setitem__(self, key, value):
        self.__dict__[key] = value
      


# Load U-net model
the final version, from file. no training here

In [12]:
#Metrics
def TP(y_true, y_pred):
    return K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
def TP_FN(y_true, y_pred):
    return K.sum(K.round(K.clip(y_true, 0, 1)))
def TP_FP(y_true, y_pred):
    return K.sum(K.round(K.clip(y_pred, 0, 1)))
def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision
def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall
def weight_loss(y_true, y_pred, weights):
    return K.binary_crossentropy(y_true, y_pred) * weights


def wrapped_partial(func, *args, **kwargs):
    partial_func = partial(func, *args, **kwargs)
    update_wrapper(partial_func, func)
    return partial_func
def conv2d_block(input_tensor, n_filters, kernel_size=3, batchnorm=True):
    # first layer
    x = Conv2D(filters=n_filters, kernel_size=(kernel_size, kernel_size), kernel_initializer="he_normal",
               padding="same")(input_tensor)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation("relu")(x)
    
    # second layer
    x = Conv2D(filters=n_filters, kernel_size=(kernel_size, kernel_size), kernel_initializer="he_normal",
               padding="same")(x)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation("relu")(x)
    return x
def unet3(n_filters, dropout=0.5, batchnorm=True, 
          input_size = (512,512,32),pretrained_weights = None, weight_size = (512,512)):
    
    input_layer = Input(input_size,name='image')
    weights_tensor = Input(weight_size,name='weights')
    
    #CONV -> BATCH -> ACTIVATION -> POOL -> DROPOUT
        # contracting path
    c1 = conv2d_block(input_layer, n_filters=n_filters*1, kernel_size=3, batchnorm=batchnorm)
    p1 = MaxPooling2D((2, 2)) (c1)
    p1 = Dropout(dropout*0.5)(p1)

    c2 = conv2d_block(p1, n_filters=n_filters*2, kernel_size=3, batchnorm=batchnorm)
    p2 = MaxPooling2D((2, 2)) (c2)
    p2 = Dropout(dropout)(p2)

    c3 = conv2d_block(p2, n_filters=n_filters*4, kernel_size=3, batchnorm=batchnorm)
    p3 = MaxPooling2D((2, 2)) (c3)
    p3 = Dropout(dropout)(p3)

    c4 = conv2d_block(p3, n_filters=n_filters*8, kernel_size=3, batchnorm=batchnorm)
    p4 = MaxPooling2D(pool_size=(2, 2)) (c4)
    p4 = Dropout(dropout)(p4)
    
    c5 = conv2d_block(p4, n_filters=n_filters*16, kernel_size=3, batchnorm=batchnorm)
    p5 = MaxPooling2D(pool_size=(2, 2)) (c5)
    p5 = Dropout(dropout)(p5)
    
    c6 = conv2d_block(p5, n_filters=n_filters*32, kernel_size=3, batchnorm=batchnorm)
    p6 = MaxPooling2D(pool_size=(2, 2)) (c6)
    p6 = Dropout(dropout)(p6)
    
    #--------------
    #8x8
    c7 = conv2d_block(p6, n_filters=n_filters*64, kernel_size=3, batchnorm=batchnorm)
    #4x4
    
    #-------------------
    
    # expansive path
    u8 = Conv2DTranspose(n_filters*32, (3, 3), strides=(2, 2), padding='same') (c7)
    u8 = concatenate([u8, c6])
    u8 = Dropout(dropout)(u8)
    c8 = conv2d_block(u8, n_filters=n_filters*32, kernel_size=3, batchnorm=batchnorm)

    u9 = Conv2DTranspose(n_filters*16, (3, 3), strides=(2, 2), padding='same') (c8)
    u9 = concatenate([u9, c5])
    u9 = Dropout(dropout)(u9)
    c9 = conv2d_block(u9, n_filters=n_filters*16, kernel_size=3, batchnorm=batchnorm)

    u10 = Conv2DTranspose(n_filters*8, (3, 3), strides=(2, 2), padding='same') (c9)
    u10 = concatenate([u10, c4])
    u10 = Dropout(dropout)(u10)
    c10 = conv2d_block(u10, n_filters=n_filters*8, kernel_size=3, batchnorm=batchnorm)

    u11 = Conv2DTranspose(n_filters*4, (3, 3), strides=(2, 2), padding='same') (c10)
    u11 = concatenate([u11, c3], axis=3)
    u11 = Dropout(dropout)(u11)
    c11 = conv2d_block(u11, n_filters=n_filters*4, kernel_size=3, batchnorm=batchnorm)   
    
    u12 = Conv2DTranspose(n_filters*2, (3, 3), strides=(2, 2), padding='same') (c11)
    u12 = concatenate([u12, c2], axis=3)
    u12 = Dropout(dropout)(u12)
    c12 = conv2d_block(u12, n_filters=n_filters*2, kernel_size=3, batchnorm=batchnorm)  
    
    u13 = Conv2DTranspose(n_filters*1, (3, 3), strides=(2, 2), padding='same') (c12)
    u13 = concatenate([u13, c1], axis=3)
    u13 = Dropout(dropout)(u13)
    c13 = conv2d_block(u13, n_filters=n_filters*1, kernel_size=3, batchnorm=batchnorm)  
    
    ans = Conv2D(1, 1, activation = 'sigmoid', name='output')(c13)
    
    cl4 = wrapped_partial(weight_loss, weights=weights_tensor)
    
    model = Model(input = [input_layer, weights_tensor], output = ans)
    
    #optimizer = Adam(lr = 1e-4)
    #optimizer = SGD(lr=1e-4, momentum=0.1, decay=0.0, nesterov=False)
    #optimizer =SGD(lr=0.001, momentum=0.5, decay=0.0, nesterov=True)
    optimizer = Adagrad()
    
    model.compile(optimizer = optimizer, loss = cl4, 
                  metrics = [precision,recall, TP, TP_FN, TP_FP])
    
    if(pretrained_weights):
    	model.load_weights(pretrained_weights)
        
    return model

In [13]:
model_path = os.path.join(args.model_path, 'unet_new_v19_ADAGRAD.hdf5')

sess = tf.Session()
K.set_session(sess)
model = unet3(pretrained_weights = model_path, 
              n_filters = 32, 
              input_size = (512,512,args['channels_N']))

# Prepare input files
using all params from arg variable (section 2)

In [89]:
############################################################
#  A Class for dataset generation. It read all data to samples structure.
############################################################
class Dataset():
    def __init__(self, size=0, mode='train'):
        self.size = size
        self.mode = mode #train or test
        self.mode_path = 'test_ero/' if mode != 'train' else 'train_ero/'
        self.samples = []
        self.dfs_n = []
        self.sam_df = {} 
        self.dfs = {}
        #Path work
        
        #self.stat = pd.read_excel('unet_v1_stat.xlsx')
        #self.stat = self.stat.iloc[0:0]  
        
        hdulist = fits.open(args['wcs_file'])
        self.my_wcs = wcs.WCS(hdulist[0].header)
        
    # Full data creation
    def prepare(self, save_to_disk = True): #saves all data to samples
        self.prepare_telescope()
        for i in range(self.size): #per dfs     
            all_sample_found = True
            self.dfs_n.append(i)
            self.sam_df[i] = []
            n = i
            
            #Read data     
            self.ph_ero_file = args.ph_ero_file % i
            self.ph_evt_file = args.ph_evt_file % i
            self.simput_file = args.simput_file % i
            self.noize_file = args.noize_file % i

            print('Current ERO', args.ph_ero_file % i)
            print('Current EVT', args.ph_evt_file % i)
            print('Current SIMPUT', args.simput_file % i)
            print('Current NOIZE', args.noize_file % i)
            
            #don't need to read all data if all samples are saved on disk already
            all_sample_found = False
            df, simput = self.read_batch_data(n)
            self.dfs[i] = df
            
            #Read frames from file!
            
            if self.mode == 'test':
                print('Reading test frames')
                frames = self.get_all_frames(df)
                #frames = [Frame(**dct) for dct in np.load(args['all_frames_file'])]
            else:
                print('Reading train frames')
                frames = [Frame(**dct) for dct in np.load(args['valid_frames_file'])]
                
            self.frames = frames
            
            print ("FRAMES IN THIS DF", len(frames))

            for j, frame in enumerate(frames):
                sample_file = args.dataset_file % (i, j)
                
                self.sam_df[i].append(len(self.samples))
                
                if os.path.isfile(sample_file): #we already have this file!
                    print(len(self.samples), str(frame), 'found')
                    s = Sample(n = n, frame_i = j, mode = self.mode)
                    s.load_from_file(sample_file)
                    
                else: #have to count
                    #if at least one sample in not found - than we have to read all data
                    if all_sample_found:
                        df, simput = self.read_batch_data() 
                        all_sample_found = False
                        
                    print(len(self.samples), str(frame), 'not found')
                    s = Sample(df = df, simput = simput,frame = frame, n = n, frame_i = j, mode = self.mode)
                    s.prepare()
                    if save_to_disk:
                        s.save_to_file(file = sample_file)
                
                self.samples.append(s)
                
                #Steps
                if len(self.samples) >= self.size:
                    return 
    def data_generator(self, save_to_disk = True, start_from = 0): #same as for prepare, but generator
        self.prepare_telescope()
        start = 0
        for i in range(len(self.dfs_names)): #per dfs   
            if start < start_from: #skip while not get start from
                start += 25
                continue
                
            all_sample_found = True
            n = re.findall(r'\d+', self.sim_names[i])[0]
            self.dfs_n.append(n)
            self.sam_df[n] = []
            
            #Read data
            df_path  = self.cur_photon_path + self.dfs_names[i]  
            sim_path = self.cur_simput_path + self.sim_names[i]
            print('Current DF', df_path)
            print('Current SIMPUT', sim_path)
            #don't need to read all data if all samples are saved on disk already
            #df, simput = self.read_batch_data(df_path, sim_path, min_flux = args['min_flux'])
            
            #Read frames from file!
            frames = [Frame(**dct) for dct in np.load(args['frames_path'])]
            #frames = self.get_valid_frames(df)
            print ("FRAMES IN THIS DF", len(frames))

            for j, frame in enumerate(frames):
                sample_file = self.data_dir+"df_{n}_frame_{j}.npy".format(n=n,j=j)
                self.sam_df[n].append(len(self.samples))
                
                if os.path.isfile(sample_file): #we already have this file!
                    print(len(self.samples), str(frame), 'found')
                    s = Sample()
                    s.load_from_file(sample_file)
                    
                else: #have to count
                    #if at least one sample in not found - than we have to read all data
                    if all_sample_found:
                        df, simput = self.read_batch_data(df_path, sim_path, min_flux = args['min_flux']) 
                        all_sample_found = False
                        
                    print(len(self.samples), str(frame), 'not found')
                    s = Sample(df, simput, frame)
                    s.prepare()
                    if save_to_disk:
                        s.save_to_file(file = sample_file)
                    #self.samples.append(s.generate_data(mode = self.mode))

                yield s.generate_data(mode = self.mode)        
                    
    #  Data reading and preprocessing
    def read_batch_data(self, n):
        df = self.prepocess_photons(ffile = self.ph_ero_file, ero = True)
        simput = self.read_simput(ffile = self.simput_file)
        
        simput['EXT'] = np.zeros(len(simput))
        
        if args.join_cluster and args.data_type != 'cluster':
            print(f'Adding cluster data to {args.data_type}')
            
            #reselect files
            self.ph_ero_file = args.cl_ph_ero_file % n
            self.ph_evt_file = args.cl_ph_evt_file % n
            self.simput_file = args.cl_simput_file % n
            
            df_cl = self.prepocess_photons(ffile = self.ph_ero_file, ero = True)
            simput_cl = self.read_simput(ffile = self.simput_file)
            simput_cl['EXT'] = np.ones(len(simput_cl))
            simput_cl['SRC_ID'] = simput_cl['SRC_ID'].apply(lambda src_id: src_id*1000000)
            df_cl = df_cl.query('SRC_ID!=-1')
            df_cl['SRC_ID'] = df_cl['SRC_ID'].apply(lambda src_id: src_id*1000000)

            self.debug_simput = simput_cl
            self.debug_df = df_cl
            
            print(f'CLUST LEN {len(simput_cl)}')
            
            #what if src_id are same?
            df = pd.concat([df, df_cl])
            simput = pd.concat([simput, simput_cl])
            
            print(f"CLUST LEN {len(simput.query('EXT == 1'))}")
            
        print('CURRENT MIN FLUX', args.min_flux)
        df,simput = self.cut_data(df, simput, args.min_flux,ret_simput = True)      
        self.simput = simput
        self.df = df
                  
        print(f"CLUST LEN {len(simput.query('EXT == 1'))}")
    
        return df, simput     

    ############################################################
    #  Low level processing
    ############################################################
    def read_photons(self, ffile, ero = False):
        #Data reading
        with fits.open(ffile, memmap=True) as hdu:
            data = hdu[1].data

        df = pd.concat([
            pd.DataFrame(data.field("RA").byteswap().newbyteorder(),columns=['RA']),
            pd.DataFrame(data.field("DEC").byteswap().newbyteorder(),columns=['DEC']),
            #pd.DataFrame(data.field("RAWX").byteswap().newbyteorder(),columns=['RAWX']), #new
           # pd.DataFrame(data.field("RAWY").byteswap().newbyteorder(),columns=['RAWY']), #new
            #pd.DataFrame(data.field("PHA").byteswap().newbyteorder(),columns=['PHA']),
            pd.DataFrame(data.field("TIME").byteswap().newbyteorder(),columns=['TIME'])
        ], axis=1)


        if ero:
            df = pd.concat([df, 
                pd.DataFrame(data.field("ENERGY").byteswap().newbyteorder(),columns=['SIGNAL'])
         ], axis = 1)     

        else: #not in evt files we also have ph and src ids'
            df = pd.concat([df, 
                pd.DataFrame(data.field("SIGNAL").byteswap().newbyteorder(),columns=['SIGNAL']),
                pd.DataFrame(data.field("PH_ID").byteswap().newbyteorder(),columns=['PH_ID', 'ZERO_COL'])['PH_ID'],
                pd.DataFrame(data.field("SRC_ID").byteswap().newbyteorder(),columns=['SRC_ID','ZERO_COL'])['SRC_ID']
         ], axis = 1)


        return df   
    def prepocess_photons(self, ffile, ero = False, read_evt = True):
        df = self.read_photons(ffile, ero)
        
        #Delete src with photons < limit and PHA == -1
        if args['verbose'] and not(ero):
            print('Read full df')
            print('На всей картинке, объектов:' , str(len(df['SRC_ID'].unique())), 'Фотонов: ', len(df))  
        if args['remove_null_pha']:
            df = df[df['PHA']!=-1]
            if args['verbose'] and not(ero):
                print('На всей картинке после удаления PHA -1  объектов:' , str(len(df['SRC_ID'].unique())), 'Фотонов: ', len(df))  
        if args['photon_limit_cut'] and not(ero):
            photon_n = df.groupby(['SRC_ID']).count().reset_index()
            true_objects = list(photon_n[photon_n['RA'] > args['photon_limit']]['SRC_ID'].unique())
            df = df[df['SRC_ID'].isin(true_objects)]

        #Adds x,y to df #Use WCS params from one of the erbox images
        df = self.projectXY(df)    
        
        df = df[['RA', 'DEC','SIGNAL', 'TIME', 'X', 'Y']]
        
        
        df = self.add_telectope_dist(df)
        
        print('FIRST READ',len(df))
        
        noize_df = self.add_noize(df) #creates
        if not(args['add_noize']): #it is easier to cut df then to remove all this code
            noize_df = noize_df[:1]
        
        
        #noize_df = self.add_telectope_dist(noize_df)
        noize_df['OFFSET'] = sample_from_distribution(df['OFFSET'], len(noize_df))

        print('FIRST READ',len(df))
        #if ero is true than we should also read evt and get src_id column there.
        if ero:
            print('CURRENT EVT', self.ph_evt_file)
            evt = self.read_photons(self.ph_evt_file)
            
            df,noize_df = self.merge_ero_evt(df,evt,noize_df)
        
        print('SEC READ',len(df))
        df = pd.concat([df, noize_df], axis = 0)

        
        #my custom scale SIGNAL ----> PHA and OFFSET ----> OFFSET_CHAN
        scale = interp1d([500, 2000], [1,args['pha_channels_N']])
        df['PHA'] = df['SIGNAL'].apply(lambda signal: int(np.round(scale(signal))))
        scale = interp1d([df['OFFSET'].min(),df['OFFSET'].max()], [1,args['offset_channels_N']])
        df['OFFSET_CHAN'] = df['OFFSET'].apply(lambda offset: int(np.round(scale(offset))))
        df['PHOTON_TYPE'] = (df['PHA'] * df['OFFSET_CHAN']).astype(int)
        

        
        self.df =df.copy()
        df = df[['RA', 'DEC','PHA', 'TIME', 'OFFSET', 'OFFSET_CHAN', 'X', 'Y', 'SRC_ID','PHOTON_TYPE']]
        return self.cut_to_sky_size(df)
    def cut_to_sky_size(self,df):
        #df = df[(df.X >=10) & (df.X <=2690) & (df.Y >=10) & (df.Y <=2690)]
        #df['X'] = df['X'] - min(df['X'])
        #df['Y'] = df['Y'] - min(df['Y'])
        return df
    def merge_ero_evt(self,ero,evt,noize):
        def f1 (x):
            return float("%.4f" % x)
        def f2 (x):
            num = str(x-min_time).split('.')
            return int(num[0])# + '.' + num[1][:2])


        #want to make a good conversion.
        ero['DEC'] = ero['DEC'].apply(f1)
        ero['RA'] = ero['RA'].apply(f1)
        evt['DEC'] = evt['DEC'].apply(f1)
        evt['RA'] = evt['RA'].apply(f1)
        noize['DEC'] = noize['DEC'].apply(f1)
        noize['RA'] = noize['RA'].apply(f1)
        
        min_time = 629434995
        evt['TIME'] = evt['TIME'].apply(f2)
        ero['TIME'] = ero['TIME'].apply(f2)
        noize['TIME'] = noize['TIME'].apply(f2)
        
        merge = ero.merge(evt[['TIME', 'RA', 'DEC', 'SRC_ID']], on =['RA', 'DEC','TIME'], how = 'inner')
        print('ERO LEN', len(ero))
        print('EVT LEN', len(evt))
        print('MERGE LEN', len(merge))

        return merge, noize
    def projectXY (self,df):
        #Get wcs from random image
        #print('WCS FROM', args['wcs_file'])

        #According to mesch
        assert (abs(self.my_wcs.wcs.cdelt[0] * self.my_wcs.wcs.cdelt[1] * 3600 * 1000) < 9)
        #print ('PIXEL SIZE', abs(w.wcs.cdelt[0] * w.wcs.cdelt[1] * 3600 * 1000))

        df = df[(df['RA'] >= 0) & (df['RA']<= 3)][ (df['DEC'] >= 0) & (df['DEC']<= 3) ]

        df['X'], df['Y'] = self.my_wcs.all_world2pix(df['RA'], df['DEC'], 1) 
        
        #df = df[(df.X >=10) & (df.X <=2690) & (df.Y >=10) & (df.Y <=2690)]
        df = df.query(f'{args.P0} <= X < {args.P0 + args.sky_size} and {args.P0} <= Y < {args.P0 + args.sky_size}')
        
        #For absolute sky
        #df['X'] -= df['X'].min()
        #df['Y'] -= df['Y'].min()

        df['X'] = df['X'].astype(np.int16)
        df['Y'] = df['Y'].astype(np.int16)

        return df
    def read_simput(self,ffile):
        with fits.open(ffile, memmap=True) as hdu:
            #hdu.info()
            data = hdu[1].data
            #spec = hdu[3].data

        #Skipped IMGROTA, IMGSCALE, SPECTRUM
        simput = pd.concat([
            pd.DataFrame(data.field("SRC_ID").byteswap().newbyteorder(),columns=['SRC_ID']),
            pd.DataFrame(data.field("RA").byteswap().newbyteorder(),columns=['RA']),
            pd.DataFrame(data.field("DEC").byteswap().newbyteorder(),columns=['DEC']),
            pd.DataFrame(data.field("E_MIN").byteswap().newbyteorder(),columns=['E_MIN']),
            pd.DataFrame(data.field("E_MAX").byteswap().newbyteorder(),columns=['E_MAX']),
            pd.DataFrame(data.field("FLUX").byteswap().newbyteorder(),columns=['FLUX']),
        ], axis=1)

        simput['LOG_FLUX'] = np.log10(simput['FLUX'])
        simput['FLUX_CLASS'] = pd.cut(simput['LOG_FLUX'], bins = np.linspace(-14.7, -11.8, 11), labels = np.arange(1,11))
        return self.cut_to_sky_size(self.projectXY(simput))
    def prepare_telescope(self): #read and sort data
        print('Telescope data preparing')
        #READ DATA
        def read_telescope(ffile):
            #Data reading
            with fits.open(ffile, memmap=True) as hdu:
                data = hdu[1].data

            df = pd.concat([
                pd.DataFrame(data.field("TIME").byteswap().newbyteorder(),columns=['TIME']),
                pd.DataFrame(data.field("RA").byteswap().newbyteorder(),columns=['RA']),
                pd.DataFrame(data.field("DEC").byteswap().newbyteorder(),columns=['DEC']),
            ], axis=1)

            return df

        tel = pd.DataFrame()
        for ffile in sorted(os.listdir(args['telescope_path'])):
            #print(ffile)
            tmp = read_telescope(args['telescope_path'] + ffile)
            tel = pd.concat([tel,tmp], axis=0)

        #asof needs sorted data
        tel = tel.sort_values('TIME').reset_index()
        tel['TEL_TIME'] = tel['TIME']
        tel['TIME'] = tel['TIME'].astype(float)
        self.tel = tel 
    def add_telectope_dist (self, df):
        df =  df.sort_values('TIME').reset_index()
        df['PH_TIME'] = df['TIME']
        
        df['TIME'] = df['TIME'].astype(float)

        df = pd.merge_asof(df, self.tel, on='TIME') \
                .rename(columns = {'RA_x':'RA', 'DEC_x':'DEC','RA_y':'TEL_RA', 'DEC_y':'TEL_DEC'}) \
                .drop(['index_y', 'index_x', 'level_0'], axis = 1, errors = 'ignore') 

        #Finally count distance
        def dist(row):
            return math.acos(math.cos(90-row['DEC']) * math.cos(90 - row['TEL_DEC']) \
                            + math.sin(90-row['DEC']) * math.sin(90 - row['TEL_DEC']) * math.cos(row['RA']-row['TEL_RA']))

        df['OFFSET'] = df.apply(dist, axis = 1)
        return df
    def add_noize(self,df):
        print('Noize preparing')
        noize = fits.open(self.noize_file)[1].data
        
        

        max_time, min_time = df['TIME'].max(), df['TIME'].min()
        #min_sig, max_pha = df['PHA'].min(), df['PHA'].max()
        time = list(df['TIME'])
        
        scale = interp1d([1,args['pha_channels_N']], [500,2000])

        new_rows = []
        for i in range(noize.shape[0]):
            for j in range(noize.shape[1]):
                for k in range(noize[i][j]):
                    ph = {}
                    ph['X'] = j #придется порпавить, это не мои картинки же
                    ph['Y'] = i

                    m = random.randint(0, len(df)-1)
                    ph['TIME'] = df['TIME'].iloc[m]
                    ph['RA'],ph['DEC'] = self.my_wcs.all_pix2world(ph['X'], ph['Y'],1)


                    ph['SIGNAL'] = random.random() * 1500 + 500
                    ph['SRC_ID'] = -1

                    new_rows.append(ph)    
        
        noize_df = pd.DataFrame(new_rows)
        return noize_df
        #return pd.concat([df, pd.DataFrame(new_rows)], axis = 0)
    
    # flux < conf limit = noize -1
    # conf limit < flux < min_flux - throw out
    def cut_data(self, df_full, simput_full, min_flux, ret_simput = False): #Remove sources with flux < threshold
        print('NOIZE PHOTONS N (before)',len(df_full[df_full['SRC_ID'] == -1]))
        print(f"$$$$ DF CHECK {len(df_full.query('SRC_ID == 800'))}")
        df = df_full.copy()

        #Select object with flux < conf limit and set them to noize
        self.simput_full = simput_full.copy()
        noize_obj = list(simput_full[simput_full['FLUX'] < args['conf_limit']]['SRC_ID'])
            
        df.loc[df['SRC_ID'].isin(noize_obj), 'SRC_ID'] = -1 #SET THEM TO NOIZE
        print(f"$$$$ DF CHECK {len(df.query('SRC_ID == 800'))}")
              
        #Remove objects with flux < min_flux
        print(f"--cut data CLUST LEN {len(simput_full.query('EXT == 1'))}")      
        simput = simput_full[simput_full['FLUX'] >= args['conf_limit']]
        print(f"--cut data CLUST LEN {len(simput.query('EXT == 1'))}") 
        print('--EXAMPLE', df.iloc[0]['SRC_ID'])
              
        good_obj = list(simput['SRC_ID']) + [-1]
        df = df[df['SRC_ID'].isin(good_obj)] 
        print(f"$$$$ DF CHECK {len(df.query('SRC_ID == 800'))}")
        
        #Remove objects from simput which don't have photons in df
        good_obj = list(set(list(df['SRC_ID'])))
        simput = simput[simput['SRC_ID'].isin(good_obj)] 
        print(f"--cut data CLUST LEN {len(simput.query('EXT == 1'))}") 
              
        print(len(simput_full),"objects overall; ", len(simput), "objects with flux more than", min_flux)
        noize_n = len(df[df['SRC_ID'] == -1])
        print('NOIZE PHOTONS N (after)', noize_n)
        print('OBJECTS PHOTONS N (after)',len(df) - noize_n)
        return df,simput
    def cut_simput_flux(self, simput, flux):
        return simput[simput['FLUX'] >= flux]

    ############################################################
    #  Frame cut work
    ############################################################
    def get_valid_frames(self,df): #Get without borders
        sizeX = sizeY = 512
        X,Y=sorted(df['X'].astype(int)),sorted(df['Y'].astype(int)) 
        xmin,xmax = X[int(len(X)*0.01)], X[int(len(X)*(1-0.01))]
        ymin,ymax = Y[int(len(Y)*0.01)], Y[int(len(Y)*(1-0.01))]
        n_tiles = int(min((xmax-xmin)/sizeX, (ymax-ymin)/sizeY))
        frames = []
        for cellX in range(n_tiles):
            for cellY in range(n_tiles):
                x0,y0 = xmin+cellX*sizeX, ymin+cellY*sizeY
                x1,y1 = x0+sizeX, y0+sizeY
                frames.append(Frame(x0,x1,y0,y1))

        return frames
    def get_all_frames(self,df): #Get all images with borders
        sizeX = sizeY = 512
        xmin,xmax = df['X'].min(), df['X'].max()
        ymin,ymax = df['Y'].min(), df['Y'].max()
        n_tiles = int((xmax-xmin)/sizeX + 1)
        frames = []
        for cellX in range(n_tiles):
            for cellY in range(n_tiles):
                x0,y0 = xmin+cellX*sizeX, ymin+cellY*sizeY
                x1,y1 = x0+sizeX, y0+sizeY
                if x1 > args['sky_size']:
                    x1 = args['sky_size']
                    x0 = x1 - sizeX
                if y1 > args['sky_size']:
                    y1 = args['sky_size']
                    y0 = y1 - sizeY


                frames.append(Frame(x0,x1,y0,y1))

        return frames
    
    ############################################################
    #  Statistics
    ############################################################
    def stat_simput_photons_per_object(self):
        return self.samples[0].df.groupby('SRC_ID').agg({'RA': 'count'}). \
                                     reset_index().groupby('RA').count().reset_index(). \
                                     rename(columns = {'RA':'PHOTONS_IN_SRC', 'SRC_ID':'N_OBJECTS'}). \
                                     sort_values('PHOTONS_IN_SRC')
    def stat_flux_from_photonons_per_object(self): #not works
        #d с помощью groupby от df
        d['NEW_RA'] = d['RA'].apply(lambda ra: (ra // 3) * 3)
        means = d.groupby('NEW_RA').mean().reset_index()
        #plt.xscale("log")
        plt.figure(figsize=(60,8))
        plt.yscale("log")
        plt.xticks(rotation=90)
        plt.xlabel('N photons')
        sns.boxplot(x = 'NEW_RA', y = 'FLUX', data = d)
        sns.lineplot(x = [0, 300], y =[data_gen_args['min_flux'],data_gen_args['min_flux']])
        plt.savefig('11.jpg')
    def predict_all(self,model):
        for i in range(self.size):
            self.samples[i].predict(model) 
    def merge_stats(self, model,model_path='unet'): #merge all stats from samples #merge stats from all samples. work just for one flux now
        stats = []
        for i in range(self.size):
            self.samples[i].get_stat(model, model_path)
            stats.append(self.samples[i].stat)
        
        self.stat = pd.concat(stats)\
               .groupby(['mode', 'thres', 'min_flux'])['TP','FP','FN','CP','PCP', 'object_pixels', 'N', 'all_ph', 'found_ph']\
               .sum().reset_index()

        #Stat processing
        self.stat = self.stat[self.stat['thres'] != 1]
        self.stat['prec'] = self.stat['TP'] / self.stat['PCP'] 
        self.stat['rec'] = self.stat['TP'] / self.stat['CP'] 
        self.stat.drop_duplicates(inplace = True)
        self.stat.sort_values(by = ['mode', 'min_flux', 'thres'], inplace = True)
        self.stat['object_ratio'] = self.stat['object_pixels'] / (self.stat['N'] * 512**2)
        self.stat['object_count_fraction'] = self.stat['found_ph'] / self.stat['all_ph'] 
       
    ############################################################
    #  File work
    ############################################################
    def save_to_file(self,file='ds_test.npy'):
        np.save(file, self.__dict__)
    def load_from_file(self, file): #init from dictionary
        ds_dict = np.load(file)[()]
        for k, v in ds_dict.items():
            setattr(self, k, v)
    def save_stat(self, file = "unet_results/unet10_gauss7.xlsx" ):
        self.stat.to_excel(file)

    #Here we save everything to boxlist.fits
    def save_boxlists(self,  d = args['nn_res_path']):
        #create detected tables for each sample
        for i in tqdm_notebook(range(self.size)): 
            self.samples[i].boxlist()
        #concatenate detected tables from samples to big boxlists
        create_dir(args.nn_res_path)
        for df_n, sam_ind in self.sam_df.items():
            self.df_detected = pd.concat([self.samples[i].detected for i in sam_ind], axis = 0)   
            self.save_boxlist(self.df_detected, d+f'/boxlist_{df_n}.fits')
    def save_boxlist(self,src, file):
            Nsrcs = len(src)
            k = args['save_boxlist_k']
            #Three columns to identify source #+
            #c1   = fits.Column(name='id_src', array=np.repeat(np.arange(Nsrcs)+1.,k).astype(int), format='1J')
            #c2   = fits.Column(name='id_inst', array=np.array([0,1,1]*Nsrcs), format='1J')
            #c3   = fits.Column(name='id_band', array=np.array([0,0,1]*Nsrcs), format='1J')
            # we will assume that net_counts is somewhat equivalent to scts
            #c4   = fits.Column(name='scts',   array=np.repeat(src['SRC_CTS'],k),     format='1E')
            #c5   = fits.Column(name='scts_err', array=np.repeat(src['SRC_CTS_ERR'],k), format='1E')
            #c6   = fits.Column(name='box_cts',   array=np.repeat(src['BOX_CTS'],k),     format='1E')
            #convert RA,DEC from wavcat to x,y values
            #c7   = fits.Column(name='x_ima',   array=np.repeat(src['X'],k),     format='1E')
            #c8   = fits.Column(name='x_ima_err', array=np.array([0.1]*Nsrcs*k), format='1E')
            #c9   = fits.Column(name='y_ima',   array=np.repeat(src['Y'],k),     format='1E')
            #c10  = fits.Column(name='y_ima_err', array=np.array([0.1]*Nsrcs*k), format='1E')
            #likelihood = -np.log10(erfc(wavcathdus[1].data['SRC_SIGNIFICANCE']))
            #print (likelihood[:5], wavcathdus[1].data['SRC_SIGNIFICANCE'][:5])
            c11  = fits.Column(name='DET_LIKE_0', array=np.repeat(src['LIKE'],k), format='1E')
            #c12  = fits.Column(name='bg_map', array=np.repeat(src['BKG_CTS'],k), format='1E')
            #c13  = fits.Column(name='bg_raw', array=np.repeat(src['BKG_CTS'],k), format='1E')
            #c14  = fits.Column(name='exp_map', array=np.repeat(src['MEAN_EXP_TIME'],k), format='1E')
            c15  = fits.Column(name='ML_FLUX_0', array=np.repeat(src['FLUX'],k), format='1E')
            c16  = fits.Column(name='ML_FLUX_ERR_0', array=np.repeat(src['FLUX_ERR'],k), format='1E')
            #c17  = fits.Column(name='rate', array=np.repeat(src['NET_RATE'],k), format='1E')
            #c18  = fits.Column(name='rate_err', array=np.repeat(src['NET_RATE_ERR'],k), format='1E')
            c19  = fits.Column(name='RA', array=np.repeat(src['RA'],k), format='1E')
            c20  = fits.Column(name='DEC', array=np.repeat(src['DEC'],k), format='1E')
            #c21  = fits.Column(name='RADEC_ERR', array=np.repeat(src['RA_DEC_ERR'],k), format='1E')
            #c22  = fits.Column(name='lII', array=np.array([0.]*Nsrcs*k), format='1E')
            #c23  = fits.Column(name='bII', array=np.array([0.]*Nsrcs*k), format='1E')
            #c24  = fits.Column(name='hr1', array=np.array([0.]*Nsrcs*k), format='1E')
            #c25  = fits.Column(name='hr1_err', array=np.array([-999.9]*Nsrcs*k), format='1E')
            #c26  = fits.Column(name='hr2', array=np.array([0.]*Nsrcs*k), format='1E')
            #c27  = fits.Column(name='hr2_er', array=np.array([-999.9]*Nsrcs*k), format='1E')
            ##c28  = fits.Column(name='hr3', array=np.array([0.]*Nsrcs*k), format='1E')
            #c29  = fits.Column(name='hr3_err', array=np.array([-999.9]*Nsrcs*k), format='1E')
            #c30  = fits.Column(name='vignetting', array=np.array([0.]*Nsrcs*k), format='1E')

            #we need to estimate box_size from 3-sigma ellipse
            # box_size = wavcathdus[1].data['R'].T
            # mean_r   = 0.5*(box_size[0]+box_size[1])
            # mean_r   = ((mean_r*0.05)/80)/3.
            #c31  = fits.Column(name='box_size', array=np.repeat(src['BOX_SIZE'],k).astype(int), format='1J')
            #c32  = fits.Column(name='eef', array=np.array([0.9]*Nsrcs*k), format='1E')
            #c33  = fits.Column(name='dist_nn', array=np.array([0.]*Nsrcs*k), format='1E')

            #EXT ZEROS
            c34  = fits.Column(name='EXT', array=np.array([0.]*Nsrcs*k), format='1E')
            #c35  = fits.Column(name='EXT_ERR', array=np.array([0.]*Nsrcs*k), format='1E')
            #c36  = fits.Column(name='EXT_LIKE', array=np.array([0.]*Nsrcs*k), format='1E')

            #colz = [c1,c2,c3,c4,c5,c6,c7,c8,c9,c10,
            #        c11,c12,c13,c14,c15,c16,c17,c18,c19,c20,
            #        c21,c22,c23,c24,c25,c26,c27,c28,c29,c30,
            #        c31,c32,c33,c34,c35,c36]
            colz = [c11,c15,c16,c19,c20,c34]

            header = fits.open(args.boxlist_file, memmap=True)[0].header #take it from other boxlist.fits

            newprimary = fits.PrimaryHDU(header=header)
            newcolumns = fits.BinTableHDU.from_columns(colz, header=header)
            newcat = fits.HDUList([newprimary,newcolumns])
            newcat.writeto(file, clobber=True)   
        
    #for general work ------------------
    def group_ans(self):
        self.ans = {}

        for df_n, sam_ind in self.sam_df.items():
            cur_ans = np.zeros(shape = (args.sky_size, args.sky_size))
            
            for ind in sam_ind:
                frame = self.samples[ind].frame
                cur_ans[frame.x0:frame.x1,frame.y0:frame.y1] = self.samples[ind].ans
            
            self.ans[df_n] = cur_ans.copy()
    def save_boxlists2(self):
        for df_n, cur_ans in self.ans.items():
            src = self.boxlist(cur_ans, df_n)
            self.cur_src =src
            self.save_boxlist(src, args.nn_res_path+f'/boxlist_{df_n}.fits')
    
    def boxlist(self, cur_ans, df_n):
        #Input: 2D numpy array data 
        #Output: list of blobs (blob - a list of points)
        def get_blobs(data0, pmap): #data0 - centers, pmap - probability map
            data = data0.copy()

            blob,blobs = [],[]
            sizeX, sizeY = data.shape

            def get_near(x,y,data):
                ans = []
                for dx in [-1,0,1]:
                    for dy in [-1,0,1]:
                        nx,ny = x+dx,y+dy
                        #Check if it is my center point or it's just less probable
                        if (nx >=0 and nx < sizeX and ny>=0 and ny < sizeY) and data[nx][ny]!=2 and \
                            ((data[nx][ny]==1) or (pmap[x][y] >= pmap[nx][ny] >= args['min_object_p'])): #or it's just less probable

                            data[nx][ny]=2
                            ans.append((nx,ny))

                if (x,y) in ans:
                    ans.remove((x,y))
                return ans

            def array_search(x,y,data):
                ans,cur = [],[]

                if data[x][y] == 1:
                    cur.append((x,y))

                while cur!=[]:
                    ans += cur
                    prev = cur.copy()
                    cur = []

                    for pnt in prev:
                        cur+=get_near(*pnt, data)

                return ans 

            for x in range(sizeX):
                for y in range(sizeY):
                    if data[x][y] == 1:
                        blob = []
                        blob+=array_search(x,y,data) #it is appending to blob
                        blobs.append(blob)
                        #print('Blob',blobs)
            return blobs #fitst method just to add initial blobs    

        #------get some blobs from ans----------
        centers = cur_ans.copy()
        pmap = cur_ans.copy()

        pmap = gaussian_filter(pmap, sigma = 1)

        centers = (centers == maximum_filter(centers,footprint=np.ones((9,9)))) * (pmap > args['min_center_p'])
        centers = convolve(centers, np.ones((3,3)), mode='constant', cval=0.0)
        centers = centers.astype(int)

        blobs = get_blobs(centers, pmap)
        print('All blobs', len(blobs))
        #cut small blobs
        blobs = [blob for blob in blobs if len(blob) > args['min_blob_pixels']]
        print('After min blob pixels cut', len(blobs))

        
        
        #-------count noize mean for later use
        df = self.dfs[df_n].copy()

        def is_noize(row):
            return pmap[int(row['X'])][int(row['Y'])] < args['noize_bound']

        df['is_noize'] = df.apply(is_noize, axis = 1)
        df = df.query('is_noize == True')

        S = (pmap < args['noize_bound']).astype(int).sum() #n of noize pixels
        noize_p = len(df) / S
        print('current noize p',noize_p)
        
        #-------convert blobs to dataframe src with all information
        #OK, now we have some blobs. Extract center and flux from each blob know
        src = pd.DataFrame(columns=['FLUX', 'X', 'Y', 'SRC_CTS', 'BKG_CTS'])

        #for each blob we need to count number of photons in it 
        df = self.dfs[df_n].copy()
        small_blob_N = 0
        for blob in blobs:
            cur_blob = np.array(blob).T
            x0,x1,y0,y1 = cur_blob[0].min(), cur_blob[0].max(), cur_blob[1].min(), cur_blob[1].max()
            cur_df = df[df['X'].between(x0,x1) & df['Y'].between(y0,y1)]

            X = cur_blob[0]
            Y = cur_blob[1]
            div = X / Y
            res = X - Y
            ans = df.query('X in @X and Y in @Y and X/Y in @div')

            if len(ans) < args['min_photons_in_blob']:
                small_blob_N +=1
                continue

            blob_pixels = len(blob)
            bkg_cts_count = int(noize_p * blob_pixels)
            cts = len(ans)
            src_cts = cts - bkg_cts_count #len(ans[ans['SRC_ID']!=-1])
            bkg_cts_correct = len(ans[ans['SRC_ID']==-1]) #need to estimate from image

            box_cts = len(cur_df)
            box_size = int((x1-x0+y1-y0+2)/2)
            net_rate = src_cts / cts
            
            #need to conver to absolute
            x,y = int(ans['X'].mean()),int(ans['Y'].mean())
            like = pmap[x][y] #may not probability?
            
            x+=args.P0
            y+=args.P0
            
            #x,y = self.frame.to_absolute(x+args.shift_blobs,y+args.shift_blobs) #+10 because of cut_to_sky_df.
            ra,dec = ds.my_wcs.all_pix2world(x,y,0)

            
            flux =  args['mean_ECF'] * (src_cts - bkg_cts_count) / args['exp_k']

            src = src.append({'FLUX' : flux, 'X' : x, 'Y' : y, 'RA' : ra, 'DEC' : dec,
                        'BLOB_PIXELS' : blob_pixels,
                        'CTS': cts, 'BOX_CTS' : box_cts,  'SRC_CTS' : src_cts,
                        'BKG_CTS_CORRECT' : bkg_cts_correct, 'BOX_SIZE' : box_size, 
                        'BKG_CTS_COUNT' : bkg_cts_count,
                        'LIKE' : like, 'MEAN_EXP_TIME' : args['exp_k'], 'NET_RATE' : net_rate},
                         ignore_index=True)
        
        
        print(f'BLOBS WITH < N photons {small_blob_N}')
        print(f'SRC LEN {len(src)}')
              
        #ADD ERR - just for format
        src['SRC_CTS_ERR'] = np.zeros(len(src))
        src['FLUX_ERR'] = np.zeros(len(src))
        src['NET_RATE_ERR'] = np.zeros(len(src))
        src['RA_DEC_ERR'] = np.zeros(len(src))
        src = src.sort_values(by = 'FLUX', ascending = False)

        return src
    def boxlist0(self, cur_ans):
        #Input: 2D numpy array data 
        #Output: list of blobs (blob - a list of points)
        def get_blobs(data0, pmap): #data0 - centers, pmap - probability map
            data = data0.copy()

            blob,blobs = [],[]
            sizeX, sizeY = data.shape

            def get_near(x,y,data):
                ans = []
                for dx in [-1,0,1]:
                    for dy in [-1,0,1]:
                        nx,ny = x+dx,y+dy
                        #Check if it is my center point or it's just less probable
                        if (nx >=0 and nx < sizeX and ny>=0 and ny < sizeY) and data[nx][ny]!=2 and \
                            ((data[nx][ny]==1) or (pmap[x][y] >= pmap[nx][ny] >= args['min_object_p'])): #or it's just less probable

                            data[nx][ny]=2
                            ans.append((nx,ny))

                if (x,y) in ans:
                    ans.remove((x,y))
                return ans

            def array_search(x,y,data):
                ans,cur = [],[]

                if data[x][y] == 1:
                    cur.append((x,y))

                while cur!=[]:
                    ans += cur
                    prev = cur.copy()
                    cur = []

                    for pnt in prev:
                        cur+=get_near(*pnt, data)

                return ans 

            for x in range(sizeX):
                for y in range(sizeY):
                    if data[x][y] == 1:
                        blob = []
                        blob+=array_search(x,y,data) #it is appending to blob
                        blobs.append(blob)
                        #print('Blob',blobs)
            return blobs #fitst method just to add initial blobs    

        #------get some blobs from ans----------
        #plt.imshow(cur_ans)
        #plt.show()
        
        centers = cur_ans.copy()
        pmap = cur_ans.copy()
        pmap = gaussian_filter(pmap, sigma = 1)

        print ('CUR MIN BLOB PIX', args['min_blob_pixels'])
        centers = (centers == maximum_filter(centers,footprint=np.ones((9,9)))) * (pmap > args['min_blob_pixels'])
        centers = convolve(centers, np.ones((3,3)), mode='constant', cval=0.0)
        centers = centers.astype(int)

        blobs = get_blobs(centers, pmap)
        print('All blobs', len(blobs))
        #cut small blobs
        blobs = [blob for blob in blobs if len(blob) > args['min_blob_pixels']]
        print('After min blob pixels cut', len(blobs))

        #-------count noize mean for later use
        df = self.df.copy()

        def is_noize(row):
            return pmap[int(row['X'])][int(row['Y'])] < args['noize_bound']

        df['is_noize'] = df.apply(is_noize, axis = 1)
        df = df.query('is_noize == True')

        S = (pmap < args['noize_bound']).astype(int).sum() #n of noize pixels
        noize_p = len(df) / S
        print('current noize p',noize_p)


        #-------convert blobs to dataframe src with all information
        #OK, now we have some blobs. Extract center and flux from each blob know
        src = pd.DataFrame(columns=['FLUX', 'X', 'Y', 'SRC_CTS', 'BKG_CTS'])

        #for each blob we need to count number of photons in it 
        df = self.df
        small_blob_N = 0
        for blob in tqdm_notebook(blobs):
            cur_blob = np.array(blob).T
            x0,x1,y0,y1 = cur_blob[0].min(), cur_blob[0].max(), cur_blob[1].min(), cur_blob[1].max()
            cur_df = df[df['X'].between(x0,x1) & df['Y'].between(y0,y1)]

            X = cur_blob[0]
            Y = cur_blob[1]
            div = X / Y
            res = X - Y
            ans = cur_df.query('X in @X and Y in @Y and X/Y in @div')

            if len(ans) < 5:
                small_blob_N +=1
                continue

            blob_pixels = len(blob)
            bkg_cts_count = int(noize_p * blob_pixels)
            cts = len(ans)
            src_cts = cts - bkg_cts_count #len(ans[ans['SRC_ID']!=-1])
            bkg_cts_correct = len(ans[ans['SRC_ID']==-1]) #need to estimate from image

            box_cts = 0 #len(cur_df)
            box_size = int((x1-x0+y1-y0+2)/2)
            net_rate = src_cts / cts
            
            #need to conver to absolute
            x,y = int(ans['X'].mean()),int(ans['Y'].mean())
            like = pmap[x][y] #may not probability?
            
            x,y = x+10,y+10 #+10 because of cut_to_sky_df.
            ra,dec = ds.my_wcs.all_pix2world(x,y,0)

            
            flux =  args['mean_ECF'] * (src_cts - bkg_cts_count) / args['exp_k']

            src = src.append({'FLUX' : flux, 'X' : x, 'Y' : y, 'RA' : ra, 'DEC' : dec,
                        'BLOB_PIXELS' : blob_pixels,
                        'CTS': cts, 'BOX_CTS' : box_cts,  'SRC_CTS' : src_cts,
                        'BKG_CTS_CORRECT' : bkg_cts_correct, 'BOX_SIZE' : box_size, 
                        'BKG_CTS_COUNT' : bkg_cts_count,
                        'LIKE' : like, 'MEAN_EXP_TIME' : args['exp_k'], 'NET_RATE' : net_rate},
                         ignore_index=True)
            
        print(f'BLOBS WITH < 5 photons {small_blob_N}')

        #ADD ERR - just for format
        src['SRC_CTS_ERR'] = np.zeros(len(src))
        src['FLUX_ERR'] = np.zeros(len(src))
        src['NET_RATE_ERR'] = np.zeros(len(src))
        src['RA_DEC_ERR'] = np.zeros(len(src))
        src = src.sort_values(by = 'FLUX', ascending = False)

        return src

In [85]:
###################Sample##################################
#  A Class for one sample (image) processing. Gets all nessesary data for NN
############################################################    
class Sample():
    def __init__(self, df=None, simput=None, frame=None, n = None, frame_i = None, mode = 'train'): #df + simput
        self.df = df
        self.simput = simput
        self.frame = frame
        self.n = n
        self.frame_i = frame_i
        self.kde_kernel = 'gaussian'
        self.flux_class_list = [] #in order of kde
        self.bad_KDE_src = []
        
        self.thres_list = args['unet_thres_list']
        self.thres_data = {}
        self.mode = mode
        #self.stat = pd.read_excel('unet_v1_stat.xlsx')
        #self.stat = self.stat.iloc[0:0]        
        
    def prepare(self):
        #Croping by frame
        self.df = self.df.reset_index()
        self.df, self.simput = self.frame.crop_data(self.df, self.simput)
        
        
        self.get_bin_flux()
        self.n_obj = len(self.simput)
        
        
        self.get_image_matrix() 
        
        
        self.prepare_images()
        
        self.old_simput = self.simput.copy()
        print('wwww', len(self.simput))
        
        if self.mode != 'test':
            self.image_KDE()
        #self.draw_image(draw_mode = '126')

        self.simput = self.old_simput
    #Train Matrix
    def get_image_matrix(self):
        channels_N = args['channels_N']
        mat = np.zeros((self.frame.sizeX,self.frame.sizeY,channels_N))

        if args['full_energy_channel']:
            channels_N -= 1 #Free one channel for full energy


        #chan_size = args['max_PHA'] / channels_N 

        #self.df = self.df.reset_index()

        for i in range(len(self.df)):
            row = self.df.iloc[i]
            x = int(row['X'])
            y = int(row['Y'])
            cur_chan = int(row['PHOTON_TYPE']) - 1 #-1 because we have [1,32] values
            #cur_chan = row['PHA'] # min( int(row['PHA'] / chan_size), channels_N-1) 
            #mat[x][y][chan] += exp[x][y] #when i'll get the exp map
            mat[x][y][cur_chan] += 1

            if args['full_energy_channel']:
                mat[x][y][channels_N] += row['SIGNAL'] #just add signal to last channel

        self.mat = mat
        return 

    ############################################################
    #  Box + mask work
    ############################################################
    def normalize_point(self, x,y):
        x = min(self.frame.sizeX-1, x)
        y = min(self.frame.sizeY-1, y)
        x = max(0, x)
        y = max(0, y)

        return [int(x),int(y)]
    def has_neighbours(self, pixel_mask, x, y): #Check if >1 source are neigbours in this point
        #Work with borders
        zone = pixel_mask[max(0,x-1):min(self.frame.sizeX-1,x+1)+1, max(0,y-1):min(self.frame.sizeY-1, y+1)+1]
        src_li = set()
        for i in range(zone.shape[0]):
            for j in range(zone.shape[1]):
                if zone[i][j].src_id != None:
                    src_li.add(zone[i][j].src_id)

        return len(src_li) > 1
    def object_KDE(self, x, y, bandwidth = 3, xbins=100j, ybins=100j, **kwargs): 
        bandwidth = args['KDE_bandwidth']
        #bandwidth = 1.06 * np.std(x) * len(x) ** (-0.2)

        xy_train  = np.vstack([y, x]).T
        kde_skl = KernelDensity(bandwidth=bandwidth, kernel = self.kde_kernel, **kwargs)
        kde_skl.fit(xy_train)
        return kde_skl 
    def object_mask_pixels(self, X,Y, search_area = 100):
        #A bit sheety, but works. Normalizing data
        meanX, meanY = int(np.median(X)), int(np.median(Y))
        X = [x - meanX for x in X]
        Y = [y - meanY for y in Y]

        #Apply KDE
        kde = self.object_KDE(X, Y)

        #Get regions with p > threshold (A bit sheety)
        mask, pnt = [], []
        #for x in range(-search_area, search_area):
        #    for y in range(-search_area, search_area):
        #        pnt.append([x,y])
        
        r = list(range(-search_area, search_area))
        pnt = list(itertools.product(*[r, r]))

        mask = np.exp(kde.score_samples(pnt))
        mask = mask.reshape((len(mask),1))
        
        #print(mask)

        pnt = np.array(pnt)
        pnt = np.concatenate( (pnt,mask) , axis =1)
        #print('KDE', pnt.shape)
        pnt = pnt [list(np.where(mask > args['p_mask'])), ][0]
        #Denormalize
        pnt = [self.normalize_point(value[0]+meanX,value[1]+meanY) + [value[2]] for value in pnt]

        return pnt
    def n_obj_zone(self, delta,pixel_mask,x,y, no_bg = True):
        zone = pixel_mask[max(0,x-delta):min(self.frame.sizeX-1,x+delta)+1, max(0,y-delta):min(self.frame.sizeY-1, y+delta)+1]
        zone = set([elem.src_id for elem in zone.flatten()])
        if no_bg and None in zone:
            zone.remove(None)
        return len(zone) 
    def get_dist_to_closest(self, pixel_mask,x,y):

        if self.n_obj_zone(9,pixel_mask,x,y) < 2:
            return 9,9

        #Work with borders
        delta = 1
        d1=d2=9
        for delta in range(1,10):
            n = self.n_obj_zone(delta,pixel_mask,x,y)
            if n == 1:
                d1 = min(d1, delta)
            elif n>=2:
                d1 = min(d1, delta)
                d2 = min(d2, delta)
                return d1,d2

        return d1,d2        
    def get_box(self, pnts0, src_id):
        #pnts[i] : x,y,prob
        pnts = np.array(pnts0).T
        xmax,xmin,ymax,ymin = pnts[0,:].max(),pnts[0,:].min(),pnts[1,:].max(),pnts[1,:].min()
        box_size = max(xmax-xmin, ymax-ymin)
        x0 = int(xmin + (xmax-xmin)/2)
        y0 = int(ymin + (ymax-ymin)/2) 
        try:
            log_bin_flux = int(list(self.simput[self.simput['SRC_ID']==int(src_id)]['FLUX_CLASS'])[0])
        except:
            print('MEGA ERROR',src_id, (list(self.simput[self.simput['SRC_ID']==int(src_id)]['FLUX_CLASS'])[0])) 
            log_bin_flux = 5
        
        self.flux_class_list.append(log_bin_flux) #
        return [x0,y0,box_size,log_bin_flux]
    def remove_by_src_id(self, bad_list):
        self.df = self.df[~self.df['SRC_ID'].isin(bad_list)]
        self.simput = self.simput[~self.simput['SRC_ID'].isin(bad_list)]
        self.n_obj = len(self.simput)   
    def image_KDE(self):
        #Сделать 2D list. В нём объекты - Pixel (p, src, color). Дальше по этой карте рисовать чисто маску.
        #For images
        pic = self.photon_pic.copy()

        gt = np.zeros((self.frame.sizeX, self.frame.sizeY)) #ground truth
        weight_mask = np.ones((self.frame.sizeX, self.frame.sizeY)) #error mask

        #Prepare data
        SRC_X = {}
        SRC_Y = {}

        #Get all X,Y coordinated for each source to dict.
        for i in range(len(self.df)):
            row = self.df.iloc[i]
            src,x,y = row['SRC_ID'],row['X'],row['Y']
            if src == -1:
                continue

            if src in SRC_X.keys():
                    SRC_X[src].append(x)
                    SRC_Y[src].append(y)
            else:
                SRC_X[src] = [x]
                SRC_Y[src] = [y]
                
        self.SRC_X = SRC_X
        self.SRC_Y = SRC_Y

        #Fill KDE to matrix
        pixel_mask = np.array([[Pixel() for j in range(self.frame.sizeY)] for i in range(self.frame.sizeX)])
        
        #per source binary mask for F-CNN
        #binary_object_mask = np.zeros([self.frame.sizeX, self.frame.sizeY, len(SRC_X.keys())], dtype=np.uint8) 
        boxes = [] #box[i] = [X0,Y0,WIDTH]

        for i, src_id in enumerate(SRC_X.keys()):
            X = np.array(SRC_X[src_id]) #Эм, убрал сортировку и заработало.Не понимаю
            Y = np.array(SRC_Y[src_id])

            #All pixels with > thres probability
            pnts = self.object_mask_pixels (X,Y)
            
            if len(pnts) == 0: #BAD SOURCE
                self.bad_KDE_src.append(src_id)
                continue

            boxes.append(self.get_box(pnts, src_id))


            #Modify pixel_mask for each source. Fill with max probability or red if it's border
            for pnt in pnts:   
                x,y,newP = pnt[0], pnt[1], pnt[2]

                if pixel_mask[x][y].p < newP: #<p
                    pixel_mask[x][y].p = newP
                    pixel_mask[x][y].src_id = src_id
                   
                    #For images
                    pixel_mask[x][y].src_color = self.src_color[src_id]
                    pic[x,y,:] = self.src_color[src_id]
                    
                    gt[x][y] = 1
                    weight_mask[x][y] = args['object_weight']
                    
                    #binary_object_mask[x,y,i] = 1
                    
        #If it is a border of two objects - draw red
        if args['high_border_weight']:
            for x in range(self.frame.sizeX):
                for y in range(self.frame.sizeY):
                    if self.has_neighbours(pixel_mask, x, y):

                        pixel_mask[x][y].src_color = (255,255,255)
                        pic[x,y,:] = (255,255,255)

                        gt[x][y] = 0 #remove borders.
                        weight_mask[x][y] = 10 #no errors in borders!!!

        #New borders work - Unet weights          
        if args['border_KDE']:
            #Find border points
            r = 10
            borders = set()
            for x in range(self.frame.sizeX):
                for y in range(self.frame.sizeY):
                    #if it is border object pixel with another object in r radius
                    if gt[x][y] == 1 and self.n_obj_zone(1,pixel_mask,x,y, no_bg=False) > 1 and self.n_obj_zone(r,pixel_mask,x,y ) > 1:
                            borders |= set(list(product(list(range(max(0,x-r),min(x+r, self.frame.sizeX))), 
                                                        list(range(max(0,y-r),min(y+r, self.frame.sizeY))))))


            #Create weights for border points
            #ans = []
            for pnt in borders:
                x,y = pnt

                #Settings for mask
                w0 = 10
                sigma = 4

                #Unet weight formula
                if gt[x][y] == 0: # a border or just no object
                    d1,d2 = self.get_dist_to_closest(pixel_mask, x, y)
                    weight_mask[x][y] = max(int(w0 * math.exp(- (d1+d1)**2 / (2*sigma**2))),1)
                    #ans.append(weight_mask[x][y])
            #print(ans)
        
        #Cut binary mask lenght from object with empty KDE
        #binary_object_mask = binary_object_mask[:,:,:len(SRC_X.keys()) - len(self.bad_KDE_src)]
        #Remove such objects from df and simput
        self.remove_by_src_id(self.bad_KDE_src)
        
        self.KDE_pic = pic
        self.gt = gt 
        self.wm = weight_mask 
        #self.boxes = boxes
        #self.binary_object_mask = binary_object_mask
        return
    
    ############################################################
    #  Painting dataset
    ############################################################
    @staticmethod
    def get_distinct_colors(n):
        def HSVToRGB(h, s, v):
            (r, g, b) = colorsys.hsv_to_rgb(h, s, v)
            return (int(255*r), int(255*g), int(255*b))

        huePartition = 1.0 / (n + 1)
        return (HSVToRGB(huePartition * value, 1, 0.5) for value in range(0, n))  #Generating N different colors
    def get_bin_flux(self):
        min_flux = abs(self.simput['LOG_FLUX'].max())
        ran = abs(self.simput['LOG_FLUX'].max() - self.simput['LOG_FLUX'].min())
        n = 10
        def f(value):
            return n - int((np.float(abs(value)) - np.float(min_flux)) / ran * n)
        self.simput['LOG_FLUX_BIN']= self.simput['LOG_FLUX'].apply(f)
        return
    def prepare_images(self): #prepare mat with photonos and color_src
        #Sampling not need for this data
        all_src = list(self.df['SRC_ID'].unique())
        cur_src = all_src



        #Set color to each source
        colorN = args['n_src']
        colors = list(Sample.get_distinct_colors(colorN))
        src_color = {}
        for key in list(set(self.df['SRC_ID'])):
            src_color[key] = colors[random.randint(0,colorN-1)]

        #Create image matrix
        mat = np.zeros(shape=(self.frame.sizeX, self.frame.sizeY, 3))
        mat_wht = np.zeros(shape=(self.frame.sizeX, self.frame.sizeY, 3))

        #Fill it with data from cur_df (photons)
        for i in range(len(self.df)):
            mat[self.df['X'].iloc[i], self.df['Y'].iloc[i]] =  src_color[self.df['SRC_ID'].iloc[i]]
            mat_wht[self.df['X'].iloc[i], self.df['Y'].iloc[i]] = (125,125,125)

        #try convolution to enlarge objects
        kernel = np.ones((3,3))
        mat = cv2.filter2D(mat, cv2.CV_64F, kernel)   
        
        self.photon_pic = mat
        self.src_color = src_color
    def paint_matrix(self,mats, size = 50):
        #print (mats.shape)
        fig=plt.figure(figsize=(len(mats)*size, size))
        columns = len(mats)

        for i in range(1, columns+1):
            fig.add_subplot(1, columns, i)
            imshow(Image.fromarray(np.uint8(mats[i-1])) , interpolation='nearest')
            plt.grid = False
            #plt.axis('on')
        #plt.colorbar()
        plt.show() #matrix - list of numpy arrays.
    def save_matrix(self,mats,size = 50, path = '1.png'):
        mpl.rcParams['figure.dpi']= 300
        fig=plt.figure(figsize=(len(mats)*6, 6))
        columns = len(mats)

        for i in range(1, columns+1):
            fig.add_subplot(1, columns, i)
            plt.imshow(mats[i-1], interpolation='nearest')
            plt.grid(False)
            plt.axis('off')

        plt.savefig(path, bbox_inches='tight')
        mpl.rcParams['figure.dpi']= 80
    def draw_image (self,draw_mode = '1', **kwargs):
        #------------------------VARIOUS IMAGES MODES-------------------
        images = []

        #Draw just photons
        if '1' in draw_mode:
            images.append(self.photon_pic)

        #Draw KDE    
        if '2' in draw_mode: 
            images.append(self.KDE_pic)
        
        #Draw GT
        if '3' in draw_mode: 
            images.append(self.gt)
        
        #Draw weights
        if '4' in draw_mode: 
            images.append(self.wm)

        #Draw photons + centers
        if '5' in draw_mode:
            mat = self.photon_pic.copy()
            for i in range(len(self.simput)): #check for each source
                bin_flux = 0
                if 'LOG_FLUX_BIN' in self.simput.columns:
                    bin_flux = int(self.simput.iloc[i]['LOG_FLUX_BIN'])

                sX,sY = int(self.simput.iloc[i]['X']),int(self.simput.iloc[i]['Y'])

                #red square
                d = 5 + 2 * bin_flux
                tmp = mat[max(0,sX-d):min(self.frame.sizeX,sX+d),max(0,sY-d):min(self.frame.sizeY,sY+d)]
                for i in range(tmp.shape[0]):
                    for j in range(tmp.shape[1]):
                        #if i==j or i+j ==d
                        if i <2 or j<2 or i >= tmp.shape[0]-2 or j >= tmp.shape[1]-2:
                            tmp[i][j] = (255,255,255)

                mat[max(0,sX-d):min(self.frame.sizeX,sX+d),max(0,sY-d):min(self.frame.sizeY,sY+d)] = tmp

            images.append(mat)
        
        #Draw photons + gt boxes
        if '6' in draw_mode:
            mat = self.KDE_pic.copy()
            for box in self.boxes:
                w = int(box[2]/2)
                cv2.rectangle(mat,(box[1]-w,box[0]-w),(box[1]+w,box[0]+w),(255,255,255),1)
            images.append(mat)
            
        #Draw predicted image + centers
        if '7' in draw_mode:
            mat = np.dstack([self.ans.copy() * 255 / self.ans.max()] * 3)
            #mat[:,:,0] = self.ans.copy()
            
            for i in range(len(self.simput)): #check for each source
                bin_flux = 0
                if 'LOG_FLUX_BIN' in self.simput.columns:
                    bin_flux = int(self.simput.iloc[i]['LOG_FLUX_BIN'])

                sX,sY = int(self.simput.iloc[i]['X']),int(self.simput.iloc[i]['Y'])

                #red square
                d = 5 + 2 * bin_flux
                tmp = mat[max(0,sX-d):min(self.frame.sizeX,sX+d),max(0,sY-d):min(self.frame.sizeY,sY+d)]
                for i in range(tmp.shape[0]):
                    for j in range(tmp.shape[1]):
                        #if i==j or i+j ==d
                        if i <2 or j<2 or i >= tmp.shape[0]-2 or j >= tmp.shape[1]-2:
                            tmp[i][j] = (255,0,0)

                mat[max(0,sX-d):min(self.frame.sizeX,sX+d),max(0,sY-d):min(self.frame.sizeY,sY+d)] = tmp

            images.append(mat)
        
        if '8' in draw_mode:
            mat = kwargs['im']
            print(mat.max())
            #mat = np.dstack([mat * 255 / mat.max()] * 3) 
            #mat = mat * 255 / mat.max()
            print(mat.max())
            
            for i in range(len(self.simput)): #check for each source
                bin_flux = 0
                if 'LOG_FLUX_BIN' in self.simput.columns:
                    bin_flux = int(self.simput.iloc[i]['LOG_FLUX_BIN'])

                sX,sY = int(self.simput.iloc[i]['X']),int(self.simput.iloc[i]['Y'])

                #red square
                d = 5 + 2 * bin_flux
                tmp = mat[max(0,sX-d):min(self.frame.sizeX,sX+d),max(0,sY-d):min(self.frame.sizeY,sY+d)]
                for i in range(tmp.shape[0]):
                    for j in range(tmp.shape[1]):
                        #if i==j or i+j ==d
                        if i <2 or j<2 or i >= tmp.shape[0]-2 or j >= tmp.shape[1]-2:
                            tmp[i][j] = (255,0,0)
                            if tmp[5][5][0] != 0:
                                tmp[i][j] = (0,255,0)
#                         if i==j or i+j==2*d-2:
#                             tmp[i][j] = (255,0,0)

                mat[max(0,sX-d):min(self.frame.sizeX,sX+d),max(0,sY-d):min(self.frame.sizeY,sY+d)] = tmp

            images.append(mat)      

        if '9' in draw_mode:
            found = kwargs['found']
            mat = np.dstack([self.ans.copy() * 255 / self.ans.max()] * 3)
            
            #mat[:,:,0] = self.ans.copy()
            
            for i in range(len(self.simput)): #check for each source
                bin_flux = 0
                if 'LOG_FLUX_BIN' in self.simput.columns:
                    bin_flux = int(self.simput.iloc[i]['LOG_FLUX_BIN'])

                sX,sY = int(self.simput.iloc[i]['X']),int(self.simput.iloc[i]['Y'])

                #red square
                d = 5 + 2 * bin_flux
                tmp = mat[max(0,sX-d):min(self.frame.sizeX,sX+d),max(0,sY-d):min(self.frame.sizeY,sY+d)]
                for i in range(tmp.shape[0]):
                    for j in range(tmp.shape[1]):
                        #if i==j or i+j ==d
                        if i <2 or j<2 or i >= tmp.shape[0]-2 or j >= tmp.shape[1]-2:
                            tmp[i][j] = (0,255,0)

                mat[max(0,sX-d):min(self.frame.sizeX,sX+d),max(0,sY-d):min(self.frame.sizeY,sY+d)] = tmp
                
            for i in range(len(found)): #check for each source
                sX,sY = int(found.iloc[i]['X']),int(found.iloc[i]['Y'])

                #red cross
                d = 5
                tmp = mat[max(0,sX-d):min(self.frame.sizeX,sX+d),max(0,sY-d):min(self.frame.sizeY,sY+d)]
                for i in range(tmp.shape[0]):
                    for j in range(tmp.shape[1]):
                        if i==j or i+j==2*d-2:
                            tmp[i][j] = (255,0,0)
                mat[max(0,sX-d):min(self.frame.sizeX,sX+d),max(0,sY-d):min(self.frame.sizeY,sY+d)] = tmp

            images.append(mat)
        
            

        #c = np.concatenate(images, axis=1)
        #paint_and_save(c, '0.jpg')
        print('Painting')
        self.paint_matrix(images)
        #self.save_matrix(images)
        #self.full_image = images
        return images

    ############################################################
    #  Getting class output in different formats
    ############################################################
    def generate_data(self, mode):
        if mode=='train':
            dct = {'image': self.mat[np.newaxis, ...], 'weights': self.wm[np.newaxis, ...]}, \
                  {'output': self.gt[np.newaxis,...,np.newaxis]}
            return dct
        elif mode == 'test':
            dct = {'image': self.mat[np.newaxis, ...], 'weights': np.zeros(shape=(1, 512, 512)) }
            return dct
        elif mode == 'ROC':
            dct = {'image': self.mat[np.newaxis, ...], 'weights': self.wm[np.newaxis, ...]}
            return self.df, self.simput, dct
       
    ############################################################
    #  File work
    ############################################################
    def save_to_file(self,file='ds_test.npy'):
        tmp = self.__dict__.copy()
        del tmp['mat']
        np.save(file, tmp) #save without mat, it's heavy
    def load_from_file(self, file): #init from dictionary
        ds_dict = np.load(file)[()]
        for k, v in ds_dict.items():
            setattr(self, k, v)        
        if not('mat' in self.__dict__): #load it back
            self.get_image_matrix()        
            
        #modify weight mask 1-1-10 scheme    #!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!1
        self.wm[self.wm != 1] = args['object_weight']
    def get_self_size(self): #get size of all my fiels
        size_dict = {}
        for key,value in self.__dict__.items():
            if isinstance(value, pd.core.frame.DataFrame):
                size_dict[key] = value.memory_usage().sum()
            if isinstance(value, np.ndarray):
                size_dict[key] = value.nbytes
        print(size_dict)
        
    ############################################################
    #  Prediction and data processing
    ############################################################
    def predict(self, model):
        data = self.generate_data(mode = 'test')
        self.ans = model.predict_on_batch(data)[0,:,:,0]        
    def get_stat(self,model, model_path = 'unet'):
        data = self.generate_data(mode = 'test')
        self.ans = model.predict_on_batch(data)[0,:,:,0]
        #ans = ans.T #We transpose it in all previous methods
        
        print('STAT',len(self.stat))
        
        min_flux = args['conf_limit'] #works only for conf limit now
        self.get_thres_data_net()
        
        
        
        try:
            name = model_path.replace('.','/').split('/')[1] #take it from model path
        except:
            name = 'unet_unparsed'
        print ('name', name)
        
        self.add_stat(name, min_flux)#stat is updated 
    def update_stat(self,mode, min_flux): #update stat from current thres_data
        for thres, data in self.thres_data.items():
            d = {'mode' : mode,
                 'min_flux' : min_flux,
                 'thres': thres, 
                 'TP' : data['TP'], 
                 'FP' : data['FP'], 
                 'FN' : data['FN'], 
                 'CP' : data['CP'], 
                 'PCP' : data['PCP'],
                 'N' : data['N'],
                 'object_pixels' : data['object_pixels'],
                 'all_ph' : data['all_ph'], 
                 'found_ph' : data['found_ph'],
                 'prec' : 0, 
                 'rec' : 0}
            new = pd.DataFrame.from_dict(d,orient='index').T
            self.stat = pd.concat([self.stat,new],
                             axis = 0, ignore_index = True)
        return 
    def add_stat(self,mode, min_flux): #update stat from current thres_data
        print('STAT',len(self.stat))
        for thres, data in self.thres_data.items():
            #add new values to stat
            if len(self.stat) == 0 or \
               len(self.stat[(self.stat.thres==thres) & \
                             (self.stat['mode']=='unet') & \
                             (self.stat.min_flux==min_flux)]) == 0:     
                self.update_stat(mode, min_flux)
            #update current values
            else:
                self.stat.loc[(self.stat.thres==thres) & 
                         (self.stat.mode=='unet') &
                         (self.stat.min_flux==min_flux), ['TP','FP','FN','CP','PCP', 'object_pixels', 'N', 'all_ph', 'found_ph']] += [data[x] for x in ['TP','FP','FN','CP','PCP','object_pixels', 'N', 'all_ph', 'found_ph']]
        return 

    #Get metrics for all thresholds for one file. We just vary thresholds here
    def get_thres_data_net (self):
        self.thres_images = []
        print('THres',args['unet_thres_list'])
        for thres in args['unet_thres_list']: 
            print ('Current Threshold', thres)

            #Some prediction post processing
            #-----------------------------------------
            cur_ans = (self.ans > thres).astype(int) #Здесь просто округляем
            print(cur_ans.sum())
            #cut out small blobs
            cur_ans = self.cut_small_blobs(cur_ans)

            if args['gauss_output']:
                #gauss a bit
                kernel_size = args['gauss_size']
                kernel = np.ones((kernel_size,kernel_size))
                cur_ans = convolution2d(cur_ans, kernel) 
                cur_ans = (cur_ans >= 1).astype(int)

            tp,fp,fn,cp,pcp,precision,recall,object_pixels,obj = self.tpr_fpr(cur_ans, paint_squares = True)

            #get objects count fraction
            all_ph = len(self.df[self.df['SRC_ID']!=-1])
            self.df['found'] = self.df.apply(lambda row: (cur_ans[int(row['X'])][int(row['Y'])] == 1).astype(int), axis = 1)
            found_ph = self.df[self.df['SRC_ID']!=-1]['found'].sum()

            #draw a matrix with images
            if random.random() < args['paint_random']: # paint_random:
                #plt.imshow(cur_ans)
                #plt.show()
#               ph = draw_image(draw_mode = '023')
#               ph.append(obj)
                self.paint_matrix([obj])

            self.update_thres(thres, tp, fp, fn, cp, pcp, object_pixels, all_ph, found_ph)
            self.thres_images.append(obj)

        return 
    def update_thres(self, thres, tp, fp, fn, cp, pcp, object_pixels, all_ph, found_ph):    
        if thres not in self.thres_data:
            self.thres_data.setdefault(thres, {'TP':0,'FP':0,'FN':0,'CP':0,'PCP':0, 'object_pixels':0, 'N':0, 'all_ph' : 0, 'found_ph' : 0})

        self.thres_data[thres]['TP']  += tp
        self.thres_data[thres]['FP']  += fp
        self.thres_data[thres]['FN']  += fn
        self.thres_data[thres]['CP']  += cp
        self.thres_data[thres]['PCP'] += pcp
        self.thres_data[thres]['object_pixels'] += object_pixels
        self.thres_data[thres]['N'] += 1
        self.thres_data[thres]['object_ratio'] = self.thres_data[thres]['object_pixels'] / (self.thres_data[thres]['N'] * self.frame.sizeX**2)
        self.thres_data[thres]['all_ph'] += all_ph
        self.thres_data[thres]['found_ph'] += found_ph
        return 
    #Get metrics for one threshold and draw image
    def tpr_fpr (self, pred, paint_squares = True, kernel_size = 7): #transforms pred matrix to 0/1 according to thres, counts found
        #some structures
        found = pred.copy()
        obj = np.repeat(found[:,:,np.newaxis] * 255, 3, axis=2)

        TP = 0

        for i in range(len(self.simput)): #check for each source
            sX,sY,src,flux,flux_bin = [int(value) for value in self.simput.iloc[i][['X','Y','SRC_ID','FLUX','LOG_FLUX_BIN']].tolist()]

            if src == -1:
                continue

            #Set green color if found, red if not
            color = (255,0,0)
            if pred[sX][sY] == 1: # or near_source(pred, sY, sX): #source found!!!
                found = self.set_found_object(found, sX, sY)
                color = (0,255,0)

                TP+=1

            #Add object squares to image
            if paint_squares:
                #red square
                d = 5 + 2*flux_bin
                #tmp = np.ones(obj[max(0,sX-d):min(image.sizeX,sX+d),max(0,sY-d):min(image.sizeY,sY+d)].shape)
                tmp = obj[max(0,sX-d):min(self.frame.sizeX,sX+d),max(0,sY-d):min(self.frame.sizeY,sY+d)].copy()
                for i in range(tmp.shape[0]):
                    for j in range(tmp.shape[1]):
                        #if i==j or i+j ==d
                        if i <2 or j<2 or i >= tmp.shape[0]-2 or j >= tmp.shape[1]-2:
                            tmp[i][j] = color

                obj[max(0,sX-d):min(self.frame.sizeX,sX+d),max(0,sY-d):min(self.frame.sizeY,sY+d)] = tmp.copy()


        FN = len(self.simput) - TP #not found objects
        CP = len(self.simput) #all true objects
        FP = len(self.get_centers(found)) #FP - all objects that are left
        PCP = TP + FP #Just all found objects

        precision = recall = 0
        if PCP !=0:
            precision = TP / PCP
        if CP != 0:
            recall = TP / CP

        object_pixels = sum(sum(pred)) 

        return TP, FP, FN,CP,PCP,precision,recall,object_pixels, obj #return all metrics for cur thres for statistics
    #Input: 2D numpy array data, point (x,y) 
    #Output: same array, all points of object in (x,y) are set to 2
    def set_found_object(self, data0, x, y):
        data = data0.copy()
        sizeX, sizeY = data.shape

        def array_search(x,y,data):
            ans,cur = [],[]

            if data[x][y] == 1:
                cur.append((x,y))

            while cur!=[]:
                ans += cur
                prev = cur.copy()
                cur = []

                for pnt in prev:
                    cur+=get_near(*pnt, data)

            return ans 

        def get_near(x,y,data):
            ans = []
            for dx in [-1,0,1]:
                for dy in [-1,0,1]:
                    nx,ny = x+dx,y+dy
                    if nx >=0 and nx < sizeX and ny>=0 and ny < sizeY and data[nx][ny]==1:
                        data[nx][ny]=2
                        ans.append((nx,ny))

            if (x,y) in ans:
                ans.remove((x,y))
            return ans

        array_search(x,y,data)
        return data
    #Input: 2D numpy array data 
    #Output: list of blobs (blob - a list of points)
    def get_blobs(self, data0):
        data = data0.copy()

        blob,blobs = [],[]
        sizeX, sizeY = data.shape

        def get_near(x,y,data):
            ans = []
            for dx in [-1,0,1]:
                for dy in [-1,0,1]:
                    nx,ny = x+dx,y+dy
                    if nx >=0 and nx < sizeX and ny>=0 and ny < sizeY and data[nx][ny]==1:
                        data[nx][ny]=2
                        ans.append((nx,ny))

            if (x,y) in ans:
                ans.remove((x,y))
            return ans

        def array_search(x,y,data):
            ans,cur = [],[]

            if data[x][y] == 1:
                cur.append((x,y))

            while cur!=[]:
                ans += cur
                prev = cur.copy()
                cur = []

                for pnt in prev:
                    cur+=get_near(*pnt, data)

            return ans 

        for x in range(sizeX):
            for y in range(sizeY):
                if data[x][y] == 1:
                    blob = []
                    blob+=array_search(x,y,data) #it is appending to blob
                    blobs.append(blob)
                    #print('Blob',blobs)
        return blobs
    #Get mean centers of all 1 blobs on 1-0 image.
    def get_centers(self,data0):
        blobs = self.get_blobs(data0)
        #print('Blobs', blobs)

        centers = []
        for i in range(len(blobs)):
            blobs[i] = np.array([[x,y] for x,y in blobs[i]])
            centers.append([int(round(blobs[i].T[0].mean())), int(round(blobs[i].T[1].mean()))])

        return centers
    #Input: 2D numpy array (aka 512,512) with 1 for object pixel and 0 for non object
    #Output: same array without blobs with < blob_thres pixel in it
    def cut_small_blobs(self, data):    
        def remove_blob(data,blob):
            for pnt in blob:
                data[pnt[0]][pnt[1]] = 0
            return data

        blobs = self.get_blobs(data)

        n_cut = 0
        for blob in blobs:
            if len(blob) < args['blob_thres']:
                n_cut += 1
                data = remove_blob(data, blob)

        print ("{0} / {1} blobs cut by blob_tres = {2}".format(n_cut,len(blobs),args['blob_thres']))
        return data
    def convolution2d(self, image, kernel):
        m, n = kernel.shape
        d = m//2
        ans = image.copy()
        if (m == n):
            y, x = image.shape
            y = y - m + 1
            x = x - m + 1
            new_image = np.zeros((y,x))
            for i in range(y):
                for j in range(x):
                    new_image[i][j] = np.sum(image[i:i+m, j:j+m]*kernel)
        ans[d:-d,d:-d] = new_image
        return ans
    def update_TP(self, TP, flux):
        for i, limit in enumerate(args['flux_limit']):
            if flux > limit:
                TP[i] += 1
        return TP
    #check if there is a source in range 3 pixels - Works in Y paradigm.
    def near_source(self, pred, sX, sY):
        sizeY,sizeX = pred.shape
        x0 = max(0, sX-3)
        x1 = min (sizeX - 1, sX+4)
        y0 = max(0, sY-3)
        y1 = min(sizeY - 1, sY + 4)

        #check if at least one pixel belongs to object
        return pred[y0:y1,x0:x1].sum() >= 1 
    
    def boxlist(self):
        #Input: 2D numpy array data 
        #Output: list of blobs (blob - a list of points)
        def get_blobs(data0, pmap): #data0 - centers, pmap - probability map
            data = data0.copy()

            blob,blobs = [],[]
            sizeX, sizeY = data.shape

            def get_near(x,y,data):
                ans = []
                for dx in [-1,0,1]:
                    for dy in [-1,0,1]:
                        nx,ny = x+dx,y+dy
                        #Check if it is my center point or it's just less probable
                        if (nx >=0 and nx < sizeX and ny>=0 and ny < sizeY) and data[nx][ny]!=2 and \
                            ((data[nx][ny]==1) or (pmap[x][y] >= pmap[nx][ny] >= args['min_object_p'])): #or it's just less probable

                            data[nx][ny]=2
                            ans.append((nx,ny))

                if (x,y) in ans:
                    ans.remove((x,y))
                return ans

            def array_search(x,y,data):
                ans,cur = [],[]

                if data[x][y] == 1:
                    cur.append((x,y))

                while cur!=[]:
                    ans += cur
                    prev = cur.copy()
                    cur = []

                    for pnt in prev:
                        cur+=get_near(*pnt, data)

                return ans 

            for x in range(sizeX):
                for y in range(sizeY):
                    if data[x][y] == 1:
                        blob = []
                        blob+=array_search(x,y,data) #it is appending to blob
                        blobs.append(blob)
                        #print('Blob',blobs)
            return blobs #fitst method just to add initial blobs    

        #------get some blobs from ans----------
        centers = self.ans.copy()
        pmap = self.ans.copy()

        pmap = gaussian_filter(pmap, sigma = 1)

        centers = (centers == maximum_filter(centers,footprint=np.ones((9,9)))) * (pmap > args['min_center_p'])
        centers = convolve(centers, np.ones((3,3)), mode='constant', cval=0.0)
        centers = centers.astype(int)

        blobs = get_blobs(centers, pmap)
        print('All blobs', len(blobs))
        #cut small blobs
        blobs = [blob for blob in blobs if len(blob) > args['min_blob_pixels']]
        print('After min blob pixels cut', len(blobs))

        
        
        #-------count noize mean for later use
        df = self.df.copy()

        def is_noize(row):
            return pmap[int(row['X'])][int(row['Y'])] < args['noize_bound']

        df['is_noize'] = df.apply(is_noize, axis = 1)
        df = df.query('is_noize == True')

        S = (pmap < args['noize_bound']).astype(int).sum() #n of noize pixels
        noize_p = len(df) / S
        print('current noize p',noize_p)
        
        #-------convert blobs to dataframe src with all information
        #OK, now we have some blobs. Extract center and flux from each blob know
        src = pd.DataFrame(columns=['FLUX', 'X', 'Y', 'SRC_CTS', 'BKG_CTS'])

        #for each blob we need to count number of photons in it 
        df = self.df
        small_blob_N = 0
        for blob in blobs:
            cur_blob = np.array(blob).T
            x0,x1,y0,y1 = cur_blob[0].min(), cur_blob[0].max(), cur_blob[1].min(), cur_blob[1].max()
            cur_df = df[df['X'].between(x0,x1) & df['Y'].between(y0,y1)]

            X = cur_blob[0]
            Y = cur_blob[1]
            div = X / Y
            res = X - Y
            ans = df.query('X in @X and Y in @Y and X/Y in @div')

            if len(ans) < args['min_photons_in_blob']:
                small_blob_N +=1
                continue

            blob_pixels = len(blob)
            bkg_cts_count = int(noize_p * blob_pixels)
            cts = len(ans)
            src_cts = cts - bkg_cts_count #len(ans[ans['SRC_ID']!=-1])
            bkg_cts_correct = len(ans[ans['SRC_ID']==-1]) #need to estimate from image

            box_cts = len(cur_df)
            box_size = int((x1-x0+y1-y0+2)/2)
            net_rate = src_cts / cts
            
            #need to conver to absolute
            x,y = int(ans['X'].mean()),int(ans['Y'].mean())
            like = pmap[x][y] #may not probability?
            
            x,y = self.frame.to_absolute(x+args.shift_blobs,y+args.shift_blobs) #+10 because of cut_to_sky_df.
            ra,dec = ds.my_wcs.all_pix2world(x,y,0)

            
            flux =  args['mean_ECF'] * (src_cts - bkg_cts_count) / args['exp_k']

            src = src.append({'FLUX' : flux, 'X' : x, 'Y' : y, 'RA' : ra, 'DEC' : dec,
                        'BLOB_PIXELS' : blob_pixels,
                        'CTS': cts, 'BOX_CTS' : box_cts,  'SRC_CTS' : src_cts,
                        'BKG_CTS_CORRECT' : bkg_cts_correct, 'BOX_SIZE' : box_size, 
                        'BKG_CTS_COUNT' : bkg_cts_count,
                        'LIKE' : like, 'MEAN_EXP_TIME' : args['exp_k'], 'NET_RATE' : net_rate},
                         ignore_index=True)
        
        
        print(f'BLOBS WITH < N photons {small_blob_N}')
        print(f'SRC LEN {len(src)}')
              
        #ADD ERR - just for format
        src['SRC_CTS_ERR'] = np.zeros(len(src))
        src['FLUX_ERR'] = np.zeros(len(src))
        src['NET_RATE_ERR'] = np.zeros(len(src))
        src['RA_DEC_ERR'] = np.zeros(len(src))
        src = src.sort_values(by = 'FLUX', ascending = False)

        self.detected = src

In [20]:
###################Other##############################
#  A Class for working image processing. Mainly cutting df, arrays
############################################################          
class Frame():
    def __init__(self, x0=0,x1=0,y0=0,y1=0,sizeX=0, sizeY=0):
        self.x0 = x0
        self.x1 = x1
        self.y0 = y0
        self.y1 = y1
        self.sizeX = x1-x0
        self.sizeY = y1-y0
    def to_absolute(self, x,y):
        return x + self.x0, y + self.y0
    def crop_dataframe(self, df):
        #All pixels [x0;x1), [y0,y1)
        df = df[(df['X'] >= self.x0) & (df['X'] < self.x1) & (df['Y'] >= self.y0) & (df['Y'] < self.y1)]
        df['X'] = (df['X'] - self.x0).astype(int)
        df['Y'] = (df['Y'] - self.y0).astype(int)
        
        return df #.reset_index()  #just dataframe for image
    def crop_data(self, df, simput):
        simput = self.crop_dataframe(simput)
        df = self.crop_dataframe(df)
        
        #Remove objects which centers are not on image
        exist_obj = list(simput['SRC_ID']) + [-1]
        df = df[df['SRC_ID'].isin(exist_obj)] 

        #Remove objects which doesn't have photons in df from simput
        exist_obj = list(df['SRC_ID'])
        simput = simput[simput['SRC_ID'].isin(exist_obj)] 
        
        return df.reset_index(), simput.reset_index() #crop + delete smt
    def crop_ndarray(self, array):
        return array[self.y0:self.y1,self.x0:self.x1].copy()
    def frame_info(self):
        print('Image',self.x0,self.x1,self.y0,self.y1)
    def __repr__(self):
        return str('Frame ' + str(self.x0) +' '+ str(self.x1) +' '+ str(self.y0) +' '+ str(self.y1) )
    def load_from_dct(self, ds_dict): #init from dictionary
        #np.save('all_frames.npy', np.array([frame.__dict__ for frame in ds2.frames]))
        for k, v in ds_dict.items():
            setattr(self, k, v)
class Pixel():
    def __init__(self, p=0, src_id=None, src_color=None):
        self.p = p
        self.src_id = src_id
        self.src_color = src_color
    
############################################################
#  Utils
############################################################ 
#u
def read_fits_to_numpy (file):
    #im = read_fits_to_numpy('/home/ad.vasilchenko/Astronomy/data/last_simulations/esaas_out/90_islands.fits')
    #plt.imshow(im)
    #im.shape
    #read_fits_to_numpy(args['esaas_path']+"{n}_islands_{i}.fits".format(n=90,i=10))
    return fits.open(file)[0].data 
def get_confusion_limit():
    #soft
    a1s = 1.82
    a2s = 0.6
    S0s = 1.48 * 10 ** -14
    Ns = 6150

    S = [10 ** p for p in np.arange(-17,-12,0.02)]
    def soft(s):
        return Ns * ( ((2 * (10 ** -15))**a1s) / (s**a1s + (S0s ** (a1s-a2s)) * (s**a2s) ))

    y1 = [soft(s) for s in S]
    plt.xscale("log")
    plt.yscale("log")
    sns.lineplot(x = S, y = y1)
    sns.lineplot(x = [10**-17, 10**-13], y = [N,N])
def sample_from_distribution(dist, N):
    # Generate a KDE from the empirical sample
    sample_pdf = scipy.stats.gaussian_kde(dist)

    # Sample new datapoints from the KDE
    new_sample_data = sample_pdf.resample(N).T[:,0]
    return new_sample_data
def grid_plot(img_list):
    #Build some good grid plots
    import matplotlib as mpl
    size = math.ceil(math.sqrt(len(img_list)))
    gs = mpl.gridspec.GridSpec(size,size)
    gs.update(wspace=0.1, hspace=0.1, left=0.1, right=0.2, bottom=0.1, top=0.2)

    fig = plt.figure(figsize=(110, 110))
    #fig.suptitle('this is the figure title')

    #st = fig.suptitle("Unet Prediction with different thresholds")
    for i in range(len(img_list)):
        #plt.title('a')
        plt.subplot(gs[i])
        plt.imshow(img_list[i].astype(np.uint8))
        plt.axis('off')

    #plt.savefig('Network_Prediction.jpg')
    plt.show()
def paint_matrix(mats, size = 50):
    #print (mats.shape)
    fig=plt.figure(figsize=(len(mats)*size, size))
    columns = len(mats)

    for i in range(1, columns+1):
        fig.add_subplot(1, columns, i)
        imshow(Image.fromarray(np.uint8(mats[i-1])))# , interpolation='nearest')
        plt.grid = False
        #plt.axis('on')
    plt.colorbar()
    plt.show() #matrix - list of numpy arrays.
def create_dir(d):
    d = os.path.dirname(d)
    if not os.path.exists(d):
            os.makedirs(d)

# Create dataset from config, predict, and save data

In [97]:
args = Config(data_type = 'test', dataset_type = 'presentation', remove_old = True)

args.join_cluster = True #enable clusters
args.add_noize = False #don't want extra noize
args.conf_limit = args.flux_limit = 6e-15 #don't want to faint sources

Erasing /data/kolosovi/dataset_presentation/


In [98]:
ds = Dataset(size = 50, mode = 'test')
ds.prepare() 
ds.predict_all(model)
ds.group_ans()

Telescope data preparing
Current ERO /srg/a1/work/kate/erosita/agn/2.4.2/1e17/0_4yr_ccdall_eroevt.fits
Current EVT /srg/a1/work/kate/erosita/agn/2.4.2/1e17/0_4yr_ccdall_evt.fits
Current SIMPUT /srg/a1/work/kate/erosita/agn/2.4.2/1e17/0_agncat.simput
Current NOIZE /srg/a1/work/kate/erosita/bkg/for_sim/bkg_0.fits
FIRST READ 44510
Noize preparing
FIRST READ 44510
CURRENT EVT /srg/a1/work/kate/erosita/agn/2.4.2/1e17/0_4yr_ccdall_evt.fits
ERO LEN 44510
EVT LEN 64869
MERGE LEN 44065
SEC READ 44065
Adding cluster data to test
FIRST READ 3414
Noize preparing
FIRST READ 3414
CURRENT EVT /srg/a1/work/kate/erosita/clusters/for_rodion/0_4yr_ccdall_evt.fits
ERO LEN 3414
EVT LEN 4559
MERGE LEN 3377
SEC READ 3377
CLUST LEN 199
CLUST LEN 199
CURRENT MIN FLUX 2.3e-15
NOIZE PHOTONS N (before) 1
$$$$ DF CHECK 0
$$$$ DF CHECK 0
--cut data CLUST LEN 199
--cut data CLUST LEN 118
--EXAMPLE -1.0
$$$$ DF CHECK 0
--cut data CLUST LEN 118
114630 objects overall;  1949 objects with flux more than 2.3e-15
NOIZE PH

In [45]:
args['KDE_bandwidth'] = 3
args['p_mask'] = 0.0016 

In [100]:
#saving all boxlists to args.nn_res_path
args.nn_res_path = '/home/kolosovi/work/data/my_output/final_test'
#os.mkdir(args.nn_res_path) 

ds.save_boxlists2()

#Here are the results

os.listdir(args.nn_res_path)


All blobs 700
After min blob pixels cut 700
current noize p 0.0035795416337057774
BLOBS WITH < N photons 249
SRC LEN 451
All blobs 341
After min blob pixels cut 341
current noize p 0.005372739300534031
BLOBS WITH < N photons 105
SRC LEN 236


['boxlist_0.fits', 'boxlist_1.fits']

In [93]:
#last boxlist file
ds.cur_src

,FLUX,X,Y,SRC_CTS,BKG_CTS,BKG_CTS_CORRECT,BKG_CTS_COUNT,BLOB_PIXELS,BOX_CTS,BOX_SIZE,CTS,DEC,LIKE,MEAN_EXP_TIME,NET_RATE,RA,SRC_CTS_ERR,FLUX_ERR,NET_RATE_ERR,RA_DEC_ERR
355,1.365237e-12,1994,2532,869,NaN,0.0,0.0,229.0,914.0,17.0,869.0,2.825639103862215,1.000000,1190.29182,1.000000,0.7710214477559809,0.0,0.0,0.0,0.0
167,9.661916e-13,949,741,615,NaN,1.0,0.0,231.0,638.0,17.0,615.0,0.8361100258236619,1.000000,1190.29182,1.000000,1.9327935249465882,0.0,0.0,0.0,0.0
246,6.959722e-13,1382,170,443,NaN,0.0,0.0,236.0,462.0,17.0,443.0,0.2018818101559565,0.999997,1190.29182,1.000000,1.4516857254268971,0.0,0.0,0.0,0.0
211,3.896187e-13,1166,2320,248,NaN,2.0,0.0,238.0,262.0,18.0,248.0,2.590402445410295,0.999998,1190.29182,1.000000,1.691834211779877,0.0,0.0,0.0,0.0
27,2.450828e-13,139,1623,156,NaN,0.0,0.0,228.0,159.0,16.0,156.0,1.8156096663513364,0.999976,1190.29182,1.000000,2.8331935692655468,0.0,0.0,0.0,0.0
106,2.309434e-13,508,524,147,NaN,3.0,0.0,239.0,172.0,20.0,147.0,0.5949911623118956,0.999860,1190.29182,1.000000,2.422639636446166,0.0,0.0,0.0,0.0
151,2.073777e-13,850,1715,132,NaN,0.0,0.0,241.0,133.0,17.0,132.0,1.9182328543675242,0.999736,1190.29182,1.000000,2.043058351517957,0.0,0.0,0.0,0.0
35,1.948094e-13,164,2655,125,NaN,0.0,1.0,309.0,131.0,20.0,126.0,2.961684840151255,0.999036,1190.29182,0.992063,2.806101327199216,0.0,0.0,0.0,0.0
134,1.775279e-13,759,1015,114,NaN,2.0,1.0,294.0,122.0,22.0,115.0,1.140481303867983,0.999297,1190.29182,0.991304,2.1439836383409085,0.0,0.0,0.0,0.0
261,1.759569e-13,1445,2385,112,NaN,1.0,0.0,238.0,112.0,17.0,112.0,2.6626055625323364,0.999787,1190.29182,1.000000,1.3815702821526035,0.0,0.0,0.0,0.0
